<a href="https://colab.research.google.com/github/tdopsmang/Colab-projects/blob/main/Servicing_km_V1_0_9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Did not work on RR path, rather added bus details upto DP, in consolidated report, also batch processing done in few cases

In [3]:
!pip install --upgrade gspread
!pip install gspread google-auth-oauthlib google-auth-httplib2
!pip install gspread google-auth oauth2client
!pip install gspread oauth2client pandas matplotlib seaborn

In [4]:
from google.colab import auth
auth.authenticate_user()
import pandas as pd
import gspread
from google.auth import default
creds, _ = default()
from google.oauth2.service_account import Credentials
gc = gspread.authorize(creds)
from oauth2client.client import GoogleCredentials
from oauth2client.service_account import ServiceAccountCredentials
from datetime import datetime
from collections import defaultdict
import re
import random
import time
from collections import defaultdict
import concurrent.futures
from functools import lru_cache
import math
from concurrent.futures import ThreadPoolExecutor


In [5]:
# Details our Database spreadsheet which has details of all files, ID in Transport Department

Database_File_spreadsheet_ID = gc.open_by_key('1nJKyvV1WQmZzvbjOP7Hsp1bQJmiNsYoJOH_jIL7H9PI') #ID of Database Spreadsheet
MASsheetID = Database_File_spreadsheet_ID.worksheet('MAS')                                    #MAS Worksheet
OdometersheetID = Database_File_spreadsheet_ID.worksheet('Odometer')                          #Odometer Worksheet
ReportID = Database_File_spreadsheet_ID.worksheet('Report')                          #Odometer Worksheet

#Reports spreadsheet and worksheet details
Report_EO_SpreadsheetID = ReportID.acell('C2').value                  # Get the spreadsheet ID from cell C2
Report_EO = gc.open_by_key(Report_EO_SpreadsheetID)                    # Open RR_Oil_Lub spreadsheet
Report_EO1_worksheet_name = ReportID.acell('B2').value                        # Km after last servicing consolidated
Report_EO1_Worksheet = Report_EO.worksheet(Report_EO1_worksheet_name)  # Open RR CH engineoil worksheet

#STS RR SVP
RR_Oil_Lub_Coolant_DEF_spreadsheet_id = MASsheetID.acell('C2').value                  # Get the spreadsheet ID from cell C2
RR_Oil_Lub = gc.open_by_key(RR_Oil_Lub_Coolant_DEF_spreadsheet_id)                    # Open RR_Oil_Lub spreadsheet

RR_Engine_oil_CH_BSIV_worksheet_name = MASsheetID.acell('B2').value                        # Get the worksheet name from cell B2
engine_oil_CH_BSIV_CH_MAS_rr = RR_Oil_Lub.worksheet(RR_Engine_oil_CH_BSIV_worksheet_name)  # Open RR CH engineoil worksheet

RR_Engine_oil_CK_BSVI_worksheet_name = MASsheetID.acell('B3').value                        # Get the worksheet name from cell B3
engine_oil_CK_BSVI_CK_MAS_rr = RR_Oil_Lub.worksheet(RR_Engine_oil_CK_BSVI_worksheet_name)  # Open RR CK engineoil worksheet

RR_Report1_worksheet_name = MASsheetID.acell('B8').value                                # Get the worksheet name from report1, RR Km between and after last servicing
RR_Report1_Worksheet = RR_Oil_Lub.worksheet(RR_Report1_worksheet_name)                  # Open RR Km between and after last servicing worksheet

RR_Report1_worksheet_name = MASsheetID.acell('B8').value                                # Get the worksheet name from report1, RR Km between and after last servicing
RR_Report1_Worksheet = RR_Oil_Lub.worksheet(RR_Report1_worksheet_name)                  # Open RR Km between and after last servicing worksheet

#STS ATR SVP
ATR_Oil_Lub_Coolant_DEF_spreadsheet_id = MASsheetID.acell('C52').value                  # Get the spreadsheet ID from cell C51 ATR
ATR_Oil_Lub = gc.open_by_key(ATR_Oil_Lub_Coolant_DEF_spreadsheet_id)                    # Open ATR_Oil_Lub spreadsheet

ATR_Engine_oil_CH_BSIV_worksheet_name = MASsheetID.acell('B52').value                         # Get the worksheet name from cell B52
engine_oil_CH_BSIV_CH_MAS_ATR = ATR_Oil_Lub.worksheet(ATR_Engine_oil_CH_BSIV_worksheet_name)  # Open ATR CH engineoil worksheet

ATR_Engine_oil_CK_BSVI_worksheet_name = MASsheetID.acell('B53').value                         # Get the worksheet name from cell B52
engine_oil_CK_BSVI_CK_MAS_ATR = ATR_Oil_Lub.worksheet(ATR_Engine_oil_CK_BSVI_worksheet_name)  # Open ATR CK engineoil worksheet

#ATR_Report1_worksheet_name = MASsheetID.acell('B58').value                           #******This may not be required       # Get the worksheet name from report1, RR Km between and after last servicing
#ATR_Report1_Worksheet = ATR_Oil_Lub.worksheet(ATR_Report1_worksheet_name)              #This may not be required     # Open RR Km between and after last servicing worksheet

#STS FG
FG_Oil_Lub_Coolant_DEF_spreadsheet_id = MASsheetID.acell('C102').value                  # Get the spreadsheet ID from cell C51 ATR
FG_Oil_Lub = gc.open_by_key(FG_Oil_Lub_Coolant_DEF_spreadsheet_id)                    # Open ATR_Oil_Lub spreadsheet

FG_Engine_oil_CH_BSIV_worksheet_name = MASsheetID.acell('B102').value                         # Get the worksheet name from cell B2
engine_oil_CH_BSIV_CH_MAS_FG = FG_Oil_Lub.worksheet(FG_Engine_oil_CH_BSIV_worksheet_name)  # Open RR engineoil worksheet

FG_Engine_oil_CK_BSVI_worksheet_name = MASsheetID.acell('B103').value                         # Get the worksheet name from cell B52
engine_oil_CK_BSVI_CK_MAS_FG = FG_Oil_Lub.worksheet(FG_Engine_oil_CK_BSVI_worksheet_name)  # Open ATR CK engineoil worksheet

#FG_Report1_worksheet_name = MASsheetID.acell('B108').value                    #*****This may not be required            # Get the worksheet name from report1, RR Km between and after last servicing
#FG_Report1_Worksheet = FG_Oil_Lub.worksheet(FG_Report1_worksheet_name)          #This may not be required         # Open RR Km between and after last servicing worksheet


# Process SVP data
SVP_Odometer_id = OdometersheetID.acell('A2').value
Odometer_spreadsheet_SVP = gc.open_by_key(SVP_Odometer_id)
bus_number_SVP = sorted([worksheet.title for worksheet in Odometer_spreadsheet_SVP])
spreadsheet_name_SVP = Odometer_spreadsheet_SVP.title
start_row_svp = 2

# Process FG data
FG_Odometer_id = OdometersheetID.acell('B2').value
Odometer_spreadsheet_FG = gc.open_by_key(FG_Odometer_id)
bus_number_FG = sorted([worksheet.title for worksheet in Odometer_spreadsheet_FG])
spreadsheet_name_FG = Odometer_spreadsheet_FG.title
start_row_fg = start_row_svp + len(bus_number_SVP)

# Process Baratang data
BT_Odometer_id = OdometersheetID.acell('C2').value
Odometer_spreadsheet_BT = gc.open_by_key(BT_Odometer_id)
bus_number_BT = sorted([worksheet.title for worksheet in Odometer_spreadsheet_BT])
spreadsheet_name_BT = Odometer_spreadsheet_BT.title
start_row_bt = start_row_fg + len(bus_number_FG)

# Process Rangat data
RT_Odometer_id = OdometersheetID.acell('D2').value
Odometer_spreadsheet_RT = gc.open_by_key(RT_Odometer_id)
bus_number_RT = sorted([worksheet.title for worksheet in Odometer_spreadsheet_RT])
spreadsheet_name_RT = Odometer_spreadsheet_RT.title
start_row_rt = start_row_bt + len(bus_number_BT)

# Process Mayabunder data
MB_Odometer_id = OdometersheetID.acell('E2').value
Odometer_spreadsheet_MB = gc.open_by_key(MB_Odometer_id)
bus_number_MB = sorted([worksheet.title for worksheet in Odometer_spreadsheet_MB])
spreadsheet_name_MB = Odometer_spreadsheet_MB.title
start_row_mb = start_row_rt + len(bus_number_RT)

# Process Diglipur data
DP_Odometer_id = OdometersheetID.acell('F2').value
Odometer_spreadsheet_DP = gc.open_by_key(DP_Odometer_id)
bus_number_DP = sorted([worksheet.title for worksheet in Odometer_spreadsheet_DP])
spreadsheet_name_DP = Odometer_spreadsheet_DP.title
start_row_dp = start_row_mb + len(bus_number_MB)



Below codes are redundent, No need to run, Use this If required in future

In [6]:
# High speed, RR, code of 1.0.7 is fast but only for CH, if we incorporate CK, it will be super fast


from datetime import datetime
import math
from concurrent.futures import ThreadPoolExecutor

# List of all worksheets to process
engine_oil_worksheets = [
    'engine_oil_CH_BSIV_CH_MAS_rr',
    'engine_oil_CK_BSVI_CK_MAS_rr'
]

# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

# Function to process each worksheet
def process_worksheet(worksheet_name):
    print(f"\n🔄 Processing worksheet: {worksheet_name}")

    # Get the worksheet object
    worksheet = None
    try:
        # Assuming these are defined in your notebook environment
        worksheet = eval(worksheet_name)  # This will use the already defined variable in your notebook
    except:
        print(f"❌ Error: Worksheet '{worksheet_name}' not found or not accessible")
        return

    # Clear data in range L2:M, Odometer value in Engine Oil and Fallback date removal
    clear_range = 'L2:M'
    print(f"🧹 Clearing range {clear_range} in {worksheet_name}...")
    worksheet.batch_clear([clear_range])

    print("🔍 Starting data processing with date fallback search...")

    # 📥 Load main sheet
    print("📥 Loading worksheet data...")
    main_data = worksheet.get_all_values()
    print(f"✅ Loaded {len(main_data)-1} rows from worksheet")

    # Step 1: Collect unique (bus, date) and row mapping
    print("🔢 Mapping bus numbers and dates...")
    bus_date_map = defaultdict(set)
    row_map = {}  # maps row number → (bus, date)

    for i, row in enumerate(main_data[1:], start=2):
        try:
            date_str = row[1].strip()
            bus_num = row[4].strip()
            if date_str and bus_num:
                bus_date_map[bus_num].add(date_str)
                row_map[i] = (bus_num, date_str)
        except:
            continue

    print(f"✅ Found {len(bus_date_map)} unique bus numbers")
    print(f"✅ Found {len(row_map)} rows to process")

    # Create a cache for worksheet data to avoid redundant fetches
    bus_worksheet_cache = {}

    # Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
    print("\n🔍 Searching for exact dates or closest earlier dates...")
    bus_date_value_map = {}
    match_type_map = {}  # Tracks whether each match is exact or fallback
    fallback_date_map = {}  # Store the actual fallback date used
    success_count = 0
    fallback_count = 0

    # Function to process a single bus
    def process_bus(bus_dates_tuple):
        bus, dates = bus_dates_tuple
        sheet_success = 0
        results = []

        try:
            # Check if we already have this bus's data in cache
            if bus in bus_worksheet_cache:
                sheet_data = bus_worksheet_cache[bus]
            else:
                # Fetch the bus worksheet data and store in cache
                sheet = Odometer_spreadsheet_SVP.worksheet(bus)
                sheet_data = sheet.get_all_values()
                bus_worksheet_cache[bus] = sheet_data

            # Create a dictionary for faster exact date lookups
            date_value_lookup = {}
            for row in sheet_data[1:]:
                try:
                    if len(row) > 12:
                        date_str_key = row[1].strip()
                        date_value_lookup[date_str_key] = row[12]  # Column M
                except:
                    continue

            for date_str in dates:
                # Check for exact match using the lookup dictionary
                exact_value = date_value_lookup.get(date_str)

                if exact_value:
                    results.append((bus, date_str, exact_value, "exact", ""))
                    sheet_success += 1
                else:
                    # Try to find closest earlier date
                    value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                    if value:
                        results.append((bus, date_str, value, "fallback", fallback_date))
                        sheet_success += 1

            print(f"  ✅ Found {sheet_success}/{len(dates)} values for bus {bus}")
            return results

        except Exception as e:
            print(f"❌ Could not process sheet '{bus}': {str(e)}")
            return []

    # Process buses in parallel using ThreadPoolExecutor
    print(f"🚀 Processing {len(bus_date_map)} buses in parallel...")

    # Convert to list for ThreadPoolExecutor
    bus_dates_list = list(bus_date_map.items())

    # Use ThreadPoolExecutor for parallel processing
    # Adjust max_workers based on your system's capabilities
    with ThreadPoolExecutor(max_workers=5) as executor:
        all_results = list(executor.map(process_bus, bus_dates_list))

    # Process all results
    for bus_results in all_results:
        for bus, date_str, value, match_type, fallback_date in bus_results:
            bus_date_value_map[(bus, date_str)] = value
            match_type_map[(bus, date_str)] = match_type
            fallback_date_map[(bus, date_str)] = fallback_date

            if match_type == "exact":
                success_count += 1
            else:
                fallback_count += 1

    print(f"\n✅ Found values for {success_count} exact date matches")
    print(f"✅ Found values for {fallback_count} fallback date matches")
    print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

    # Step 3: Update values into Column L of main sheet and fallback dates into Column M
    print(f"\n📝 Preparing batch updates for {worksheet_name}...")

    # Prepare batch update for all cells at once
    batch_updates = []
    exact_match_cells = []
    fallback_match_cells = []

    for row_num, (bus, date_str) in row_map.items():
        value = bus_date_value_map.get((bus, date_str), "")
        fallback_date = fallback_date_map.get((bus, date_str), "")

        if value:
            # Add to batch updates list (row, col, value)
            batch_updates.append({
                'row': row_num,
                'col': 12,  # Column L
                'value': value
            })

            # Add fallback date update
            batch_updates.append({
                'row': row_num,
                'col': 13,  # Column M
                'value': fallback_date
            })

            # Track which cells need which formatting
            match_type = match_type_map.get((bus, date_str))
            if match_type == "exact":
                exact_match_cells.append(f"L{row_num}")
            elif match_type == "fallback":
                fallback_match_cells.append(f"L{row_num}")

    print(f"✅ Prepared {len(batch_updates)} cell updates")

    # Execute batch updates in chunks to avoid rate limits
    updated_count = 0
    chunk_size = 50  # Adjust based on API limits
    chunks = math.ceil(len(batch_updates) / chunk_size)

    print(f"🔄 Executing batch updates in {chunks} chunks...")

    for chunk_index in range(chunks):
        start_idx = chunk_index * chunk_size
        end_idx = min(start_idx + chunk_size, len(batch_updates))
        current_chunk = batch_updates[start_idx:end_idx]

        max_retries = 5
        retry_count = 0
        update_successful = False

        while not update_successful and retry_count < max_retries:
            try:
                # Prepare the batch update request
                cell_list = []

                for update in current_chunk:
                    cell = worksheet.cell(update['row'], update['col'])
                    cell.value = update['value']
                    cell_list.append(cell)

                # Execute the batch update
                worksheet.update_cells(cell_list, value_input_option='USER_ENTERED')

                update_successful = True
                updated_count += len(current_chunk) // 2  # Divide by 2 because we have 2 cells per row
                print(f"📊 Progress: Chunk {chunk_index+1}/{chunks} complete - {updated_count}/{len(batch_updates)//2} rows updated")

                # Add a small delay between chunks to avoid rate limits
                time.sleep(1)

            except Exception as e:
                retry_count += 1
                if "429" in str(e):
                    wait_time = (2 ** retry_count) + random.random()
                    print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Failed to update chunk {chunk_index+1}: {e}")
                    break

        if not update_successful:
            print(f"❌ Failed to update chunk {chunk_index+1} after {max_retries} retries")

    # Apply formatting in batches (currently commented out)
    print("\n🎨 Applying formatting to distinguish match types...")
    # You can uncomment and implement batch formatting logic if needed.

    print(f"\n✅ COMPLETE for {worksheet_name}: Updated {updated_count} rows in column L and column M")
    print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
    print(f"❌ {len(row_map) - updated_count} rows could not be updated")

# Main execution
print("🚀 Starting to process all engine oil worksheets...")

# Create a global cache for bus worksheets to reuse across all worksheets
global_bus_worksheet_cache = {}

# Process each worksheet in sequence
for worksheet_name in engine_oil_worksheets:
    print(f"\n{'='*80}\n📊 Processing worksheet: {worksheet_name}\n{'='*80}")
    process_worksheet(worksheet_name)
    # Add a delay between worksheets to avoid rate limiting
    time.sleep(2)

print("\n✅ All worksheets have been processed successfully!")
print("💡 Performance Summary:")
print(f"✓ Processed {len(engine_oil_worksheets)} worksheets with batch processing")
print(f"✓ Used parallel processing for bus data with ThreadPoolExecutor")
print(f"✓ Implemented caching to reduce API calls")
print(f"✓ Used batch updates to minimize API requests")


🚀 Starting to process all engine oil worksheets...

📊 Processing worksheet: engine_oil_CH_BSIV_CH_MAS_rr

🔄 Processing worksheet: engine_oil_CH_BSIV_CH_MAS_rr
🧹 Clearing range L2:M in engine_oil_CH_BSIV_CH_MAS_rr...
🔍 Starting data processing with date fallback search...
📥 Loading worksheet data...
✅ Loaded 946 rows from worksheet
🔢 Mapping bus numbers and dates...
✅ Found 76 unique bus numbers
✅ Found 931 rows to process

🔍 Searching for exact dates or closest earlier dates...
🚀 Processing 76 buses in parallel...
  ✅ Found 17/18 values for bus 504
  ✅ Found 12/13 values for bus 442
  ✅ Found 9/10 values for bus 530
  ✅ Found 10/11 values for bus 459
  ✅ Found 5/6 values for bus 521
❌ Could not process sheet '477': 477
❌ Could not process sheet '326': 326
❌ Could not process sheet '322': 322
❌ Could not process sheet '317': 317
❌ Could not process sheet '342': 342
  ✅ Found 33/34 values for bus 469
  ✅ Found 17/17 values for bus 508
  ✅ Found 15/15 values for bus 486
❌ Could not proces

In [7]:
# ATR , (RR, code of 1.0.7 is fast but only for CH, if we incorporate CK, and adopt for ATR and FG it will be super fast)


# List of all worksheets to process
engine_oil_worksheets = [
    'engine_oil_CH_BSIV_CH_MAS_ATR',
    'engine_oil_CK_BSVI_CK_MAS_ATR'
]

# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

# Function to process each worksheet
def process_worksheet(worksheet_name):
    print(f"\n🔄 Processing worksheet: {worksheet_name}")

    # Get the worksheet object
    worksheet = None
    try:
        # Assuming these are defined in your notebook environment
        worksheet = eval(worksheet_name)  # This will use the already defined variable in your notebook
    except:
        print(f"❌ Error: Worksheet '{worksheet_name}' not found or not accessible")
        return

    # Clear data in range L2:M, Odometer value in Engine Oil and Fallback date removal
    clear_range = 'L2:M'
    print(f"🧹 Clearing range {clear_range} in {worksheet_name}...")
    worksheet.batch_clear([clear_range])

    print("🔍 Starting data processing with date fallback search...")

    # 📥 Load main sheet
    print("📥 Loading worksheet data...")
    main_data = worksheet.get_all_values()
    print(f"✅ Loaded {len(main_data)-1} rows from worksheet")

    # Step 1: Collect unique (bus, date) and row mapping
    print("🔢 Mapping bus numbers and dates...")
    bus_date_map = defaultdict(set)
    row_map = {}  # maps row number → (bus, date)

    for i, row in enumerate(main_data[1:], start=2):
        try:
            date_str = row[1].strip()
            bus_num = row[4].strip()
            if date_str and bus_num:
                bus_date_map[bus_num].add(date_str)
                row_map[i] = (bus_num, date_str)
        except:
            continue

    print(f"✅ Found {len(bus_date_map)} unique bus numbers")
    print(f"✅ Found {len(row_map)} rows to process")

    # Create a cache for worksheet data to avoid redundant fetches
    bus_worksheet_cache = {}

    # Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
    print("\n🔍 Searching for exact dates or closest earlier dates...")
    bus_date_value_map = {}
    match_type_map = {}  # Tracks whether each match is exact or fallback
    fallback_date_map = {}  # Store the actual fallback date used
    success_count = 0
    fallback_count = 0

    # Function to process a single bus
    def process_bus(bus_dates_tuple):
        bus, dates = bus_dates_tuple
        sheet_success = 0
        results = []

        try:
            # Check if we already have this bus's data in cache
            if bus in bus_worksheet_cache:
                sheet_data = bus_worksheet_cache[bus]
            else:
                # Fetch the bus worksheet data and store in cache
                sheet = Odometer_spreadsheet_SVP.worksheet(bus)
                sheet_data = sheet.get_all_values()
                bus_worksheet_cache[bus] = sheet_data

            # Create a dictionary for faster exact date lookups
            date_value_lookup = {}
            for row in sheet_data[1:]:
                try:
                    if len(row) > 12:
                        date_str_key = row[1].strip()
                        date_value_lookup[date_str_key] = row[12]  # Column M
                except:
                    continue

            for date_str in dates:
                # Check for exact match using the lookup dictionary
                exact_value = date_value_lookup.get(date_str)

                if exact_value:
                    results.append((bus, date_str, exact_value, "exact", ""))
                    sheet_success += 1
                else:
                    # Try to find closest earlier date
                    value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                    if value:
                        results.append((bus, date_str, value, "fallback", fallback_date))
                        sheet_success += 1

            print(f"  ✅ Found {sheet_success}/{len(dates)} values for bus {bus}")
            return results

        except Exception as e:
            print(f"❌ Could not process sheet '{bus}': {str(e)}")
            return []

    # Process buses in parallel using ThreadPoolExecutor
    print(f"🚀 Processing {len(bus_date_map)} buses in parallel...")

    # Convert to list for ThreadPoolExecutor
    bus_dates_list = list(bus_date_map.items())

    # Use ThreadPoolExecutor for parallel processing
    # Adjust max_workers based on your system's capabilities
    with ThreadPoolExecutor(max_workers=5) as executor:
        all_results = list(executor.map(process_bus, bus_dates_list))

    # Process all results
    for bus_results in all_results:
        for bus, date_str, value, match_type, fallback_date in bus_results:
            bus_date_value_map[(bus, date_str)] = value
            match_type_map[(bus, date_str)] = match_type
            fallback_date_map[(bus, date_str)] = fallback_date

            if match_type == "exact":
                success_count += 1
            else:
                fallback_count += 1

    print(f"\n✅ Found values for {success_count} exact date matches")
    print(f"✅ Found values for {fallback_count} fallback date matches")
    print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

    # Step 3: Update values into Column L of main sheet and fallback dates into Column M
    print(f"\n📝 Preparing batch updates for {worksheet_name}...")

    # Prepare batch update for all cells at once
    batch_updates = []
    exact_match_cells = []
    fallback_match_cells = []

    for row_num, (bus, date_str) in row_map.items():
        value = bus_date_value_map.get((bus, date_str), "")
        fallback_date = fallback_date_map.get((bus, date_str), "")

        if value:
            # Add to batch updates list (row, col, value)
            batch_updates.append({
                'row': row_num,
                'col': 12,  # Column L
                'value': value
            })

            # Add fallback date update
            batch_updates.append({
                'row': row_num,
                'col': 13,  # Column M
                'value': fallback_date
            })

            # Track which cells need which formatting
            match_type = match_type_map.get((bus, date_str))
            if match_type == "exact":
                exact_match_cells.append(f"L{row_num}")
            elif match_type == "fallback":
                fallback_match_cells.append(f"L{row_num}")

    print(f"✅ Prepared {len(batch_updates)} cell updates")

    # Execute batch updates in chunks to avoid rate limits
    updated_count = 0
    chunk_size = 50  # Adjust based on API limits
    chunks = math.ceil(len(batch_updates) / chunk_size)

    print(f"🔄 Executing batch updates in {chunks} chunks...")

    for chunk_index in range(chunks):
        start_idx = chunk_index * chunk_size
        end_idx = min(start_idx + chunk_size, len(batch_updates))
        current_chunk = batch_updates[start_idx:end_idx]

        max_retries = 5
        retry_count = 0
        update_successful = False

        while not update_successful and retry_count < max_retries:
            try:
                # Prepare the batch update request
                cell_list = []

                for update in current_chunk:
                    cell = worksheet.cell(update['row'], update['col'])
                    cell.value = update['value']
                    cell_list.append(cell)

                # Execute the batch update
                worksheet.update_cells(cell_list, value_input_option='USER_ENTERED')

                update_successful = True
                updated_count += len(current_chunk) // 2  # Divide by 2 because we have 2 cells per row
                print(f"📊 Progress: Chunk {chunk_index+1}/{chunks} complete - {updated_count}/{len(batch_updates)//2} rows updated")

                # Add a small delay between chunks to avoid rate limits
                time.sleep(1)

            except Exception as e:
                retry_count += 1
                if "429" in str(e):
                    wait_time = (2 ** retry_count) + random.random()
                    print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Failed to update chunk {chunk_index+1}: {e}")
                    break

        if not update_successful:
            print(f"❌ Failed to update chunk {chunk_index+1} after {max_retries} retries")

    # Apply formatting in batches (currently commented out)
    print("\n🎨 Applying formatting to distinguish match types...")
    # You can uncomment and implement batch formatting logic if needed.

    print(f"\n✅ COMPLETE for {worksheet_name}: Updated {updated_count} rows in column L and column M")
    print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
    print(f"❌ {len(row_map) - updated_count} rows could not be updated")

# Main execution
print("🚀 Starting to process all engine oil worksheets...")

# Create a global cache for bus worksheets to reuse across all worksheets
global_bus_worksheet_cache = {}

# Process each worksheet in sequence
for worksheet_name in engine_oil_worksheets:
    print(f"\n{'='*80}\n📊 Processing worksheet: {worksheet_name}\n{'='*80}")
    process_worksheet(worksheet_name)
    # Add a delay between worksheets to avoid rate limiting
    time.sleep(2)

print("\n✅ All worksheets have been processed successfully!")
print("💡 Performance Summary:")
print(f"✓ Processed {len(engine_oil_worksheets)} worksheets with batch processing")
print(f"✓ Used parallel processing for bus data with ThreadPoolExecutor")
print(f"✓ Implemented caching to reduce API calls")
print(f"✓ Used batch updates to minimize API requests")


🚀 Starting to process all engine oil worksheets...

📊 Processing worksheet: engine_oil_CH_BSIV_CH_MAS_ATR

🔄 Processing worksheet: engine_oil_CH_BSIV_CH_MAS_ATR
🧹 Clearing range L2:M in engine_oil_CH_BSIV_CH_MAS_ATR...
🔍 Starting data processing with date fallback search...
📥 Loading worksheet data...
✅ Loaded 900 rows from worksheet
🔢 Mapping bus numbers and dates...
✅ Found 39 unique bus numbers
✅ Found 352 rows to process

🔍 Searching for exact dates or closest earlier dates...
🚀 Processing 39 buses in parallel...
❌ Could not process sheet '381': 381
❌ Could not process sheet '579': 579
❌ Could not process sheet '576': 576
❌ Could not process sheet '368': 368
  ✅ Found 16/16 values for bus 574
  ✅ Found 5/5 values for bus 575
  ✅ Found 14/14 values for bus 590
  ✅ Found 31/31 values for bus 546
❌ Could not process sheet '565': 565
❌ Could not process sheet '319': 319
  ✅ Found 16/16 values for bus 550
  ✅ Found 29/29 values for bus 548
  ✅ Found 3/3 values for bus 584
  ✅ Found 3/3 

In [8]:
#FG_Working, Best Code

# Modified to process multiple engine oil worksheets, Added batch processing done

# Modified to process multiple engine oil worksheets with batch processing
# With fallback date display in M Column, For ATR_MAS_Engine Oil

# List of all worksheets to process
engine_oil_worksheets = [
    'engine_oil_CH_BSIV_CH_MAS_FG',
    'engine_oil_CK_BSVI_CK_MAS_FG'
]

# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

# Function to process each worksheet
def process_worksheet(worksheet_name):
    print(f"\n🔄 Processing worksheet: {worksheet_name}")

    # Get the worksheet object
    worksheet = None
    try:
        # Assuming these are defined in your notebook environment
        worksheet = eval(worksheet_name)  # This will use the already defined variable in your notebook
    except:
        print(f"❌ Error: Worksheet '{worksheet_name}' not found or not accessible")
        return

    # Clear data in range L2:M, Odometer value in Engine Oil and Fallback date removal
    clear_range = 'L2:M'
    print(f"🧹 Clearing range {clear_range} in {worksheet_name}...")
    worksheet.batch_clear([clear_range])

    print("🔍 Starting data processing with date fallback search...")

    # 📥 Load main sheet
    print("📥 Loading worksheet data...")
    main_data = worksheet.get_all_values()
    print(f"✅ Loaded {len(main_data)-1} rows from worksheet")

    # Step 1: Collect unique (bus, date) and row mapping
    print("🔢 Mapping bus numbers and dates...")
    bus_date_map = defaultdict(set)
    row_map = {}  # maps row number → (bus, date)

    for i, row in enumerate(main_data[1:], start=2):
        try:
            date_str = row[1].strip()
            bus_num = row[4].strip()
            if date_str and bus_num:
                bus_date_map[bus_num].add(date_str)
                row_map[i] = (bus_num, date_str)
        except:
            continue

    print(f"✅ Found {len(bus_date_map)} unique bus numbers")
    print(f"✅ Found {len(row_map)} rows to process")

    # Create a cache for worksheet data to avoid redundant fetches
    bus_worksheet_cache = {}

    # Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
    print("\n🔍 Searching for exact dates or closest earlier dates...")
    bus_date_value_map = {}
    match_type_map = {}  # Tracks whether each match is exact or fallback
    fallback_date_map = {}  # Store the actual fallback date used
    success_count = 0
    fallback_count = 0

    # Function to process a single bus
    def process_bus(bus_dates_tuple):
        bus, dates = bus_dates_tuple
        sheet_success = 0
        results = []

        try:
            # Check if we already have this bus's data in cache
            if bus in bus_worksheet_cache:
                sheet_data = bus_worksheet_cache[bus]
            else:
                # Fetch the bus worksheet data and store in cache
                sheet = Odometer_spreadsheet_FG.worksheet(bus)
                sheet_data = sheet.get_all_values()
                bus_worksheet_cache[bus] = sheet_data

            # Create a dictionary for faster exact date lookups
            date_value_lookup = {}
            for row in sheet_data[1:]:
                try:
                    if len(row) > 12:
                        date_str_key = row[1].strip()
                        date_value_lookup[date_str_key] = row[12]  # Column M
                except:
                    continue

            for date_str in dates:
                # Check for exact match using the lookup dictionary
                exact_value = date_value_lookup.get(date_str)

                if exact_value:
                    results.append((bus, date_str, exact_value, "exact", ""))
                    sheet_success += 1
                else:
                    # Try to find closest earlier date
                    value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                    if value:
                        results.append((bus, date_str, value, "fallback", fallback_date))
                        sheet_success += 1

            print(f"  ✅ Found {sheet_success}/{len(dates)} values for bus {bus}")
            return results

        except Exception as e:
            print(f"❌ Could not process sheet '{bus}': {str(e)}")
            return []

    # Process buses in parallel using ThreadPoolExecutor
    print(f"🚀 Processing {len(bus_date_map)} buses in parallel...")

    # Convert to list for ThreadPoolExecutor
    bus_dates_list = list(bus_date_map.items())

    # Use ThreadPoolExecutor for parallel processing
    # Adjust max_workers based on your system's capabilities
    with ThreadPoolExecutor(max_workers=5) as executor:
        all_results = list(executor.map(process_bus, bus_dates_list))

    # Process all results
    for bus_results in all_results:
        for bus, date_str, value, match_type, fallback_date in bus_results:
            bus_date_value_map[(bus, date_str)] = value
            match_type_map[(bus, date_str)] = match_type
            fallback_date_map[(bus, date_str)] = fallback_date

            if match_type == "exact":
                success_count += 1
            else:
                fallback_count += 1

    print(f"\n✅ Found values for {success_count} exact date matches")
    print(f"✅ Found values for {fallback_count} fallback date matches")
    print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

    # Step 3: Update values into Column L of main sheet and fallback dates into Column M
    print(f"\n📝 Preparing batch updates for {worksheet_name}...")

    # Prepare batch update for all cells at once
    batch_updates = []
    exact_match_cells = []
    fallback_match_cells = []

    for row_num, (bus, date_str) in row_map.items():
        value = bus_date_value_map.get((bus, date_str), "")
        fallback_date = fallback_date_map.get((bus, date_str), "")

        if value:
            # Add to batch updates list (row, col, value)
            batch_updates.append({
                'row': row_num,
                'col': 12,  # Column L
                'value': value
            })

            # Add fallback date update
            batch_updates.append({
                'row': row_num,
                'col': 13,  # Column M
                'value': fallback_date
            })

            # Track which cells need which formatting
            match_type = match_type_map.get((bus, date_str))
            if match_type == "exact":
                exact_match_cells.append(f"L{row_num}")
            elif match_type == "fallback":
                fallback_match_cells.append(f"L{row_num}")

    print(f"✅ Prepared {len(batch_updates)} cell updates")

    # Execute batch updates in chunks to avoid rate limits
    updated_count = 0
    chunk_size = 50  # Adjust based on API limits
    chunks = math.ceil(len(batch_updates) / chunk_size)

    print(f"🔄 Executing batch updates in {chunks} chunks...")

    for chunk_index in range(chunks):
        start_idx = chunk_index * chunk_size
        end_idx = min(start_idx + chunk_size, len(batch_updates))
        current_chunk = batch_updates[start_idx:end_idx]

        max_retries = 5
        retry_count = 0
        update_successful = False

        while not update_successful and retry_count < max_retries:
            try:
                # Prepare the batch update request
                cell_list = []

                for update in current_chunk:
                    cell = worksheet.cell(update['row'], update['col'])
                    cell.value = update['value']
                    cell_list.append(cell)

                # Execute the batch update
                worksheet.update_cells(cell_list, value_input_option='USER_ENTERED')

                update_successful = True
                updated_count += len(current_chunk) // 2  # Divide by 2 because we have 2 cells per row
                print(f"📊 Progress: Chunk {chunk_index+1}/{chunks} complete - {updated_count}/{len(batch_updates)//2} rows updated")

                # Add a small delay between chunks to avoid rate limits
                time.sleep(1)

            except Exception as e:
                retry_count += 1
                if "429" in str(e):
                    wait_time = (2 ** retry_count) + random.random()
                    print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ Failed to update chunk {chunk_index+1}: {e}")
                    break

        if not update_successful:
            print(f"❌ Failed to update chunk {chunk_index+1} after {max_retries} retries")

    # Apply formatting in batches (currently commented out)
    print("\n🎨 Applying formatting to distinguish match types...")
    # You can uncomment and implement batch formatting logic if needed.

    print(f"\n✅ COMPLETE for {worksheet_name}: Updated {updated_count} rows in column L and column M")
    print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
    print(f"❌ {len(row_map) - updated_count} rows could not be updated")

# Main execution
print("🚀 Starting to process all engine oil worksheets...")

# Create a global cache for bus worksheets to reuse across all worksheets
global_bus_worksheet_cache = {}

# Process each worksheet in sequence
for worksheet_name in engine_oil_worksheets:
    print(f"\n{'='*80}\n📊 Processing worksheet: {worksheet_name}\n{'='*80}")
    process_worksheet(worksheet_name)
    # Add a delay between worksheets to avoid rate limiting
    time.sleep(2)

print("\n✅ All worksheets have been processed successfully!")
print("💡 Performance Summary:")
print(f"✓ Processed {len(engine_oil_worksheets)} worksheets with batch processing")
print(f"✓ Used parallel processing for bus data with ThreadPoolExecutor")
print(f"✓ Implemented caching to reduce API calls")
print(f"✓ Used batch updates to minimize API requests")


🚀 Starting to process all engine oil worksheets...

📊 Processing worksheet: engine_oil_CH_BSIV_CH_MAS_FG

🔄 Processing worksheet: engine_oil_CH_BSIV_CH_MAS_FG
🧹 Clearing range L2:M in engine_oil_CH_BSIV_CH_MAS_FG...
🔍 Starting data processing with date fallback search...
📥 Loading worksheet data...
✅ Loaded 999 rows from worksheet
🔢 Mapping bus numbers and dates...
✅ Found 37 unique bus numbers
✅ Found 978 rows to process

🔍 Searching for exact dates or closest earlier dates...
🚀 Processing 37 buses in parallel...
❌ Could not process sheet 'tyreplant': tyreplant
❌ Could not process sheet '388': 388
❌ Could not process sheet '419': 419
❌ Could not process sheet '389': 389
  ✅ Found 1/1 values for bus 463
❌ Could not process sheet '415': 415
  ✅ Found 34/34 values for bus 560
❌ Could not process sheet '435': 435
  ✅ Found 29/29 values for bus 507
❌ Could not process sheet 'workshop': workshop
❌ Could not process sheet '370': 370
  ✅ Found 67/67 values for bus 516
❌ Could not process shee

In [ ]:
# Lean for all units, improved API limiter, #Lean combined all Units Engine Oil and Search all Odometer, but it search only missing values, if it works, then in future just add engine oil sheet and odometer sheet


# Lean version - Only processes rows with empty L column
# With fallback date display in M Column, For SVP, FG, BT, RT, MB, and DP

import re
from datetime import datetime
import random
import time
from collections import defaultdict
import gspread
import math
from concurrent.futures import ThreadPoolExecutor

# List of all worksheets to process
engine_oil_worksheets = [
    'engine_oil_CH_BSIV_CH_MAS_FG',
    'engine_oil_CK_BSVI_CK_MAS_FG',
    'engine_oil_CH_BSIV_CH_MAS_ATR',
    'engine_oil_CK_BSVI_CK_MAS_ATR',
    'engine_oil_CH_BSIV_CH_MAS_rr',
    'engine_oil_CK_BSVI_CK_MAS_rr'
]

# List of all odometer spreadsheets to search
odometer_spreadsheets = [
    'Odometer_spreadsheet_SVP',
    'Odometer_spreadsheet_FG',
    'Odometer_spreadsheet_BT',
    'Odometer_spreadsheet_RT',
    'Odometer_spreadsheet_MB',
    'Odometer_spreadsheet_DP'
]

# Function to parse custom date format 'DD,MMMyy' like '13,Apr24'
def parse_custom_date(date_str):
    if not date_str or date_str == '':
        return None

    pattern = r'(\d{1,2}),(Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)(\d{2})'
    match = re.match(pattern, date_str)

    if match:
        day = int(match.group(1))
        month_str = match.group(2)
        year = int(match.group(3)) + 2000  # Assuming 20xx for the year

        month_map = {
            'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6,
            'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12
        }

        month = month_map[month_str]
        return datetime(year, month, day)

    return None

# Modified function to return both value and fallback date
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    # Parse all dates in the sheet
    dates = []
    parsed_dates = []
    values = []

    for row in sheet_data[1:]:  # Skip header
        try:
            if len(row) > value_column_index and len(row) > 1:
                date_str = row[1].strip()
                value = row[value_column_index]

                parsed_date = parse_custom_date(date_str)
                if parsed_date:
                    dates.append(date_str)
                    parsed_dates.append(parsed_date)
                    values.append(value)
        except:
            continue

    # Parse target date
    target_date = parse_custom_date(target_date_str)
    if not target_date:
        return None, None

    # First check for exact match
    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date == target_date:
            return values[i], None  # Return value with no fallback date for exact matches

    # If no exact match, find closest earlier date
    closest_date = None
    closest_value = None
    max_date = None

    for i, parsed_date in enumerate(parsed_dates):
        if parsed_date < target_date and (max_date is None or parsed_date > max_date):
            max_date = parsed_date
            closest_date = dates[i]
            closest_value = values[i]

    return closest_value, closest_date  # Return both value and fallback date

# Global cache for bus worksheets to reuse across all worksheets and spreadsheets
global_bus_worksheet_cache = {}

# Function to process each worksheet - LEAN VERSION (only empty L cells)
def process_worksheet_lean(worksheet_name):
    print(f"\n🔄 Processing worksheet: {worksheet_name}")

    # Get the worksheet object
    worksheet = None
    try:
        # Assuming these are defined in your notebook environment
        worksheet = eval(worksheet_name)  # This will use the already defined variable in your notebook
    except:
        print(f"❌ Error: Worksheet '{worksheet_name}' not found or not accessible")
        return

    # NO CLEARING OF RANGES - we're only updating empty cells
    print("🔍 Starting data processing for EMPTY L cells only...")

    # 📥 Load main sheet
    print("📥 Loading worksheet data...")
    main_data = worksheet.get_all_values()
    print(f"✅ Loaded {len(main_data)-1} rows from worksheet")

    # Step 1: Collect unique (bus, date) and row mapping ONLY FOR EMPTY L CELLS
    print("🔢 Mapping bus numbers and dates for empty L cells...")
    bus_date_map = defaultdict(set)
    row_map = {}  # maps row number → (bus, date)
    empty_cell_count = 0

    for i, row in enumerate(main_data[1:], start=2):
        try:
            # Check if L column (index 11) is empty
            l_value = row[11].strip() if len(row) > 11 else ""

            if not l_value:  # Only process if L column is empty
                date_str = row[1].strip()
                bus_num = row[4].strip()
                if date_str and bus_num:
                    bus_date_map[bus_num].add(date_str)
                    row_map[i] = (bus_num, date_str)
                    empty_cell_count += 1
        except:
            continue

    print(f"✅ Found {len(bus_date_map)} unique bus numbers with empty L cells")
    print(f"✅ Found {empty_cell_count} empty L cells to process")

    # Exit early if no empty cells to process
    if empty_cell_count == 0:
        print(f"✓ No empty L cells to process in {worksheet_name}. Moving to next worksheet.")
        return

    # Step 2: Fetch each bus worksheet once, collect all data and find exact or closest earlier date
    print("\n🔍 Searching for exact dates or closest earlier dates...")
    bus_date_value_map = {}
    match_type_map = {}  # Tracks whether each match is exact or fallback
    fallback_date_map = {}  # Store the actual fallback date used
    success_count = 0
    fallback_count = 0

    # Track which spreadsheet contained the match for metadata
    spreadsheet_source_map = {}  # Maps (bus, date) to spreadsheet name

    # Function to process a single bus across all spreadsheets
    def process_bus(bus_dates_tuple):
        bus, dates = bus_dates_tuple
        sheet_success = 0
        results = []
        dates_to_process = set(dates)  # Create a copy to safely modify

        # Try each spreadsheet in sequence
        for spreadsheet_name in odometer_spreadsheets:
            if not dates_to_process:  # If all dates have been found, skip further processing
                break

            try:
                # Generate cache key that includes the spreadsheet name
                cache_key = f"{spreadsheet_name}:{bus}"

                # Check if we already have this bus's data in cache
                if cache_key in global_bus_worksheet_cache:
                    sheet_data = global_bus_worksheet_cache[cache_key]
                else:
                    # Get the spreadsheet object dynamically
                    try:
                        spreadsheet = eval(spreadsheet_name)
                        # Try to get the worksheet for this bus with retry mechanism
                        max_retries = 5
                        retry_count = 0
                        while retry_count < max_retries:
                            try:
                                sheet = spreadsheet.worksheet(bus)
                                sheet_data = sheet.get_all_values()
                                global_bus_worksheet_cache[cache_key] = sheet_data
                                break  # Success, exit retry loop
                            except gspread.exceptions.APIError as api_error:
                                if hasattr(api_error, 'response') and api_error.response.status_code == 429:
                                    retry_count += 1
                                    wait_time = (2 ** retry_count) + (random.random() * 2)
                                    print(f"⏳ Rate limit hit for bus {bus}, waiting {wait_time:.2f}s (retry {retry_count}/{max_retries})")
                                    time.sleep(wait_time)
                                    if retry_count == max_retries:
                                        print(f"⚠️ Max retries reached for bus {bus} in {spreadsheet_name}")
                                        raise
                                else:
                                    # Not a rate limit issue, the worksheet likely doesn't exist
                                    raise
                            except Exception as e:
                                # The worksheet doesn't exist in this spreadsheet
                                if "Worksheet not found" in str(e) or "404" in str(e):
                                    print(f"  ℹ️ Bus {bus} not found in {spreadsheet_name}")
                                else:
                                    print(f"  ⚠️ Error accessing {bus} in {spreadsheet_name}: {str(e)}")
                                raise
                    except Exception:
                        # This bus doesn't exist in this spreadsheet or we couldn't access it
                        continue

                # Process each date for this bus in this spreadsheet
                # Create a dictionary for faster exact date lookups
                date_value_lookup = {}
                for row in sheet_data[1:]:
                    try:
                        if len(row) > 12:
                            date_str_key = row[1].strip()
                            date_value_lookup[date_str_key] = row[12]  # Column M
                    except:
                        continue

                dates_found_in_this_sheet = set()

                # First try exact matches which are faster to check
                for date_str in dates_to_process:
                    # Check for exact match using the lookup dictionary
                    exact_value = date_value_lookup.get(date_str)
                    if exact_value:
                        results.append((bus, date_str, exact_value, "exact", "", spreadsheet_name))
                        sheet_success += 1
                        dates_found_in_this_sheet.add(date_str)

                # Then try fallback matches for remaining dates
                remaining_dates = dates_to_process - dates_found_in_this_sheet
                for date_str in remaining_dates:
                    # Try to find closest earlier date
                    value, fallback_date = find_closest_date_value(sheet_data, date_str, value_column_index=12)
                    if value:
                        results.append((bus, date_str, value, "fallback", fallback_date, spreadsheet_name))
                        sheet_success += 1
                        dates_found_in_this_sheet.add(date_str)

                # Remove all processed dates
                dates_to_process -= dates_found_in_this_sheet

            except Exception as e:
                if "Worksheet not found" not in str(e) and "404" not in str(e):
                    print(f"❌ Could not process sheet '{bus}' in spreadsheet '{spreadsheet_name}': {str(e)}")
                continue

        if sheet_success > 0:
            print(f"  ✅ Found {sheet_success} values for bus {bus}")
        else:
            print(f"  ⚠️ No values found for bus {bus} in any spreadsheet")

        return results

    # Only process if we have buses to check
    if len(bus_date_map) > 0:
        # Convert to list for ThreadPoolExecutor
        bus_dates_list = list(bus_date_map.items())

        # Use ThreadPoolExecutor for parallel processing
        max_workers = min(5, len(bus_dates_list))  # Adjust based on data size
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            all_results = list(executor.map(process_bus, bus_dates_list))

        # Process all results
        for bus_results in all_results:
            for bus, date_str, value, match_type, fallback_date, source_spreadsheet in bus_results:
                bus_date_value_map[(bus, date_str)] = value
                match_type_map[(bus, date_str)] = match_type
                fallback_date_map[(bus, date_str)] = fallback_date
                spreadsheet_source_map[(bus, date_str)] = source_spreadsheet

                if match_type == "exact":
                    success_count += 1
                else:
                    fallback_count += 1
    else:
        print("⚠️ No buses to process - all L cells have values already")

    print(f"\n✅ Found values for {success_count} exact date matches")
    print(f"✅ Found values for {fallback_count} fallback date matches")
    print(f"❌ Could not find values for {len(row_map) - (success_count + fallback_count)} dates")

    # Step 3: Update values into Column L of main sheet and fallback dates into Column M
    print(f"\n📝 Preparing batch updates for {worksheet_name}...")

    # Prepare batch update for all cells at once
    batch_updates = []
    exact_match_cells = []
    fallback_match_cells = []

    for row_num, (bus, date_str) in row_map.items():
        value = bus_date_value_map.get((bus, date_str), "")
        fallback_date = fallback_date_map.get((bus, date_str), "")
        spreadsheet_source = spreadsheet_source_map.get((bus, date_str), "")

        if value:
            # Add to batch updates list (row, col, value)
            batch_updates.append({
                'row': row_num,
                'col': 12,  # Column L
                'value': value
            })

            # Add fallback date update
            batch_updates.append({
                'row': row_num,
                'col': 13,  # Column M
                'value': fallback_date if fallback_date else ""
            })

            # Add source spreadsheet info in Column N for tracking
     #       batch_updates.append({
      #          'row': row_num,
      #          'col': 14,  # Column N
      #          'value': spreadsheet_source
      #      })

            # Track which cells need which formatting
            match_type = match_type_map.get((bus, date_str))
            if match_type == "exact":
                exact_match_cells.append(f"L{row_num}")
            elif match_type == "fallback":
                fallback_match_cells.append(f"L{row_num}")

    print(f"✅ Prepared {len(batch_updates)} cell updates")

    # Skip if no updates needed
    if len(batch_updates) == 0:
        print("✓ No updates needed for this worksheet")
        return

    # Execute batch updates in chunks to avoid rate limits
    updated_count = 0
    chunk_size = 25  # Reduced chunk size to avoid API limits
    chunks = math.ceil(len(batch_updates) / chunk_size)

    print(f"🔄 Executing batch updates in {chunks} chunks...")

    for chunk_index in range(chunks):
        start_idx = chunk_index * chunk_size
        end_idx = min(start_idx + chunk_size, len(batch_updates))
        current_chunk = batch_updates[start_idx:end_idx]

        max_retries = 8  # Increased retries
        retry_count = 0
        update_successful = False

        while not update_successful and retry_count < max_retries:
            try:
                # Prepare the batch update request
                cell_list = []

                for update in current_chunk:
                    cell = worksheet.cell(update['row'], update['col'])
                    cell.value = update['value']
                    cell_list.append(cell)

                # Execute the batch update using our rate-limited function
                def do_update():
                    return worksheet.update_cells(cell_list, value_input_option='USER_ENTERED')

                rate_limited_request(do_update)

                update_successful = True
                updated_count += len(current_chunk) // 3  # Divide by 3 because we have 3 cells per row (L, M, N)
                print(f"📊 Progress: Chunk {chunk_index+1}/{chunks} complete - {updated_count}/{len(batch_updates)//3} rows updated")

                # Add a variable delay between chunks to avoid rate limits
                # More aggressive backoff with increasing chunk index
                wait_time = 2 + (chunk_index % 3) + (random.random() * 3)
                print(f"⏳ Waiting {wait_time:.2f}s before next chunk...")
                time.sleep(wait_time)

            except gspread.exceptions.APIError as api_error:
                retry_count += 1
                if hasattr(api_error, 'response') and api_error.response.status_code == 429:
                    # Exponential backoff with jitter for rate limits
                    wait_time = (2 ** retry_count) + (random.random() * 5)
                    # Cap maximum wait time at 2 minutes
                    wait_time = min(wait_time, 120)
                    print(f"⏳ Rate limit hit, waiting for {wait_time:.2f} seconds (retry {retry_count}/{max_retries})")
                    time.sleep(wait_time)
                else:
                    print(f"❌ API Error on chunk {chunk_index+1}: {api_error}")
                    if retry_count < 3:  # Only retry a few times for non-rate-limit errors
                        time.sleep(5)
                    else:
                        break
            except Exception as e:
                retry_count += 1
                print(f"❌ Failed to update chunk {chunk_index+1}: {e}")
                if retry_count < 3:  # Only retry a few times for general errors
                    time.sleep(5)
                else:
                    break

        if not update_successful:
            print(f"❌ Failed to update chunk {chunk_index+1} after {max_retries} retries")
            # Add extra delay when a chunk completely fails before moving to next chunk
            wait_time = 15 + (random.random() * 10)
            print(f"⏳ Taking a longer break: {wait_time:.2f}s before continuing...")
            time.sleep(wait_time)

    # Apply formatting in batches (currently commented out)
    print("\n🎨 Applying formatting to distinguish match types...")
    # You can uncomment and implement batch formatting logic if needed.

    print(f"\n✅ COMPLETE for {worksheet_name}: Updated {updated_count} rows in columns L, M, and N")
    print(f"📊 Summary: {success_count} exact matches (normal format), {fallback_count} fallback matches (italic and right-aligned)")
    print(f"❌ {len(row_map) - updated_count} rows could not be updated")

# Global rate limiting function
def rate_limited_request(func, *args, **kwargs):
    """Execute a function with rate limiting and retries for API errors"""
    max_retries = 8  # More retries for critical operations
    retry_count = 0
    base_wait_time = 2  # seconds

    while retry_count < max_retries:
        try:
            return func(*args, **kwargs)
        except gspread.exceptions.APIError as api_error:
            if hasattr(api_error, 'response') and api_error.response.status_code == 429:
                retry_count += 1
                # Exponential backoff with jitter
                wait_time = (base_wait_time ** retry_count) + (random.random() * 5)
                # Cap the wait time at 3 minutes
                wait_time = min(wait_time, 180)
                print(f"⏳ Rate limit hit! Waiting {wait_time:.2f}s (retry {retry_count}/{max_retries})")
                time.sleep(wait_time)
            else:
                # Re-raise non-rate-limit errors
                raise
        except Exception as e:
            # For other exceptions, retry a few times but with less patience
            if retry_count < 3:
                retry_count += 1
                wait_time = base_wait_time + (random.random() * 2)
                print(f"⚠️ Error occurred: {str(e)}")
                print(f"Retrying in {wait_time:.2f}s (retry {retry_count}/3)")
                time.sleep(wait_time)
            else:
                raise

    # If we've exhausted all retries
    raise Exception(f"Failed after {max_retries} retries due to persistent rate limiting")

# Main execution - LEAN VERSION with improved rate limiting
print("🚀 Starting LEAN processing - only empty L cells...")
print(f"🔍 Will search across {len(odometer_spreadsheets)} odometer spreadsheets: {', '.join(odometer_spreadsheets)}")

# Process each worksheet with rate limiting and proper delays
for i, worksheet_name in enumerate(engine_oil_worksheets):
    print(f"\n{'='*80}\n📊 Processing worksheet: {worksheet_name} ({i+1}/{len(engine_oil_worksheets)})\n{'='*80}")

    # Add a longer delay between worksheets to avoid rate limiting
    if i > 0:
        wait_time = 5 + (random.random() * 5)  # 5-10 second delay between worksheets
        print(f"⏳ Waiting {wait_time:.2f}s before processing next worksheet...")
        time.sleep(wait_time)

    # Process the worksheet with proper rate limiting
    try:
        process_worksheet_lean(worksheet_name)
    except Exception as e:
        print(f"❌ Error processing worksheet {worksheet_name}: {str(e)}")
        print("Continuing with next worksheet after a delay...")
        time.sleep(15)  # Longer delay after an error

print("\n✅ All worksheets have been processed successfully!")
print("💡 Performance Summary:")
print(f"✓ Processed {len(engine_oil_worksheets)} worksheets with LEAN mode (empty L cells only)")
print(f"✓ Searched across {len(odometer_spreadsheets)} odometer spreadsheets")
print(f"✓ Used parallel processing for bus data with ThreadPoolExecutor")
print(f"✓ Implemented caching to reduce API calls")
print(f"✓ Used batch updates to minimize API requests")



In [ ]:
# # High Speed, Lean for all units, improved API limiter, #Lean combined all Units Engine Oil and Search all Odometer, but it search only missing values, if it works, then in future just add engine oil sheet and odometer sheet



# The above code is returning value in Column N, i dont need it, also incorporate batch processing to increase speed

In [11]:
# If date is current date then it is returning the spreadsheet name, trying to correct, It is wokring, i will delete above code in version 1.0.9

# High speed Report Generation, in Engine Oil Report Spreadsheet

from datetime import datetime
import re
import time
from collections import defaultdict
import functools
import concurrent.futures
import threading

# === PERFORMANCE OPTIMIZATION SETUP ===
print("🚀 Initializing performance optimizations...")

# Cache for worksheet data to reduce API calls
data_cache = {}
cache_lock = threading.Lock()

# LRU Cache decorator for expensive operations
def lru_cache_with_expiry(maxsize=128, typed=False, ttl=600):
    """LRU Cache with expiry time"""
    def decorator(func):
        cache_dict = {}
        cache_times = {}
        cache_lock = threading.Lock()

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = str(args) + str(kwargs)
            current_time = time.time()

            with cache_lock:
                # Check if key exists and not expired
                if key in cache_dict and current_time - cache_times[key] < ttl:
                    return cache_dict[key]

                # Call the function and cache result
                result = func(*args, **kwargs)
                cache_dict[key] = result
                cache_times[key] = current_time

                # Remove oldest entries if cache is too large
                if len(cache_dict) > maxsize:
                    oldest_key = min(cache_times, key=cache_times.get)
                    cache_dict.pop(oldest_key)
                    cache_times.pop(oldest_key)

                return result
        return wrapper
    return decorator

# Batch processing helper functions
def chunk_list(lst, chunk_size):
    """Split list into chunks of specified size"""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

# Cached function to get worksheet data
@lru_cache_with_expiry(maxsize=100, ttl=300)  # Cache for 5 minutes
def get_worksheet_data(spreadsheet_name, worksheet_name):
    """Get worksheet data with caching"""
    cache_key = f"{spreadsheet_name}:{worksheet_name}"

    with cache_lock:
        if cache_key in data_cache:
            print(f"📋 Cache hit for {cache_key}")
            return data_cache[cache_key]

    # Function to find the actual worksheet object
    def find_worksheet(spreadsheet_list, worksheet_name, spreadsheet_names):
        for idx, spreadsheet in enumerate(spreadsheet_list):
            try:
                sheet = spreadsheet.worksheet(worksheet_name)
                print(f"📑 Found worksheet {worksheet_name} in {spreadsheet_names[idx]}")
                return sheet, spreadsheet_names[idx]
            except Exception:
                continue
        return None, None

    # Cache miss, need to retrieve data
    try:
        sheet, source = find_worksheet(odometer_spreadsheets, worksheet_name, spreadsheet_names)
        if sheet:
            data = sheet.get_all_values()

            with cache_lock:
                data_cache[cache_key] = (data, source)

            return data, source
        return None, None
    except Exception as e:
        print(f"⚠️ Error retrieving data for {worksheet_name}: {e}")
        return None, None

# === STEP 1: CLEAR DATA ===
print("\n🧹 STEP 1: Clearing data...")
clear_range_all_OdometersheetID = 'A5:R200'
clear_range_all_Report_EO1_Worksheet = ['B2:B200', 'AC2:AC200']

OdometersheetID.batch_clear([clear_range_all_OdometersheetID])
Report_EO1_Worksheet.batch_clear(clear_range_all_Report_EO1_Worksheet)

# === STEP 2: PROCESS BUS DATA ===
print("\n📊 STEP 2: Processing bus data...")

# Function to process spreadsheet data in parallel
def process_spreadsheet(cell_ref, position_index):
    """Process a single spreadsheet and return its data"""
    odometer_id = OdometersheetID.acell(cell_ref).value
    spreadsheet = gc.open_by_key(odometer_id)
    bus_numbers = sorted([worksheet.title for worksheet in spreadsheet])
    spreadsheet_name = spreadsheet.title
    return {
        'spreadsheet': spreadsheet,
        'name': spreadsheet_name,
        'buses': bus_numbers,
        'position': position_index
    }

# Process all spreadsheets in parallel
spreadsheet_info = {
    'SVP': {'cell': 'A2', 'position': 0},
    'FG': {'cell': 'B2', 'position': 1},
    'BT': {'cell': 'C2', 'position': 2},
    'RT': {'cell': 'D2', 'position': 3},
    'MB': {'cell': 'E2', 'position': 4},
    'DP': {'cell': 'F2', 'position': 5}
}

spreadsheet_data = {}
odometer_spreadsheets = [None] * 6
spreadsheet_names = [None] * 6

# For Google Sheets API, we can't fully parallelize due to rate limits
# But we can optimize the processing steps
for key, info in spreadsheet_info.items():
    data = process_spreadsheet(info['cell'], info['position'])
    spreadsheet_data[key] = data
    odometer_spreadsheets[info['position']] = data['spreadsheet']
    spreadsheet_names[info['position']] = data['name']
    print(f"✅ Processed {key} spreadsheet: {data['name']} with {len(data['buses'])} buses")

# Calculate starting rows for each section
start_row_svp = 2
start_row_fg = start_row_svp + len(spreadsheet_data['SVP']['buses'])
start_row_bt = start_row_fg + len(spreadsheet_data['FG']['buses'])
start_row_rt = start_row_bt + len(spreadsheet_data['BT']['buses'])
start_row_mb = start_row_rt + len(spreadsheet_data['RT']['buses'])
start_row_dp = start_row_mb + len(spreadsheet_data['MB']['buses'])

# === STEP 3: UPDATE ODOMETERSHEETID WITH BUS NUMBERS ===
print("\n📝 STEP 3: Updating OdometersheetID with bus numbers...")

# Optimized batch update function
def optimized_batch_update(worksheet, updates):
    """Process updates in optimal batch sizes"""
    BATCH_SIZE = 100  # Adjust based on API limits

    # Split updates into manageable chunks
    chunked_updates = chunk_list(updates, BATCH_SIZE)

    for chunk in chunked_updates:
        try:
            worksheet.batch_update(chunk)
            time.sleep(0.5)  # Slight delay to avoid rate limiting
        except Exception as e:
            print(f"⚠️ Batch update error: {e}")
            # Fall back to smaller chunks if needed
            for update in chunk:
                try:
                    worksheet.update(update['range'], update['values'])
                    time.sleep(0.1)
                except Exception as e2:
                    print(f"⚠️ Individual update error at {update['range']}: {e2}")

# Update spreadsheet names in row 4 (batch update)
name_updates = [
    {'range': 'A4', 'values': [[spreadsheet_data['SVP']['name']]]},
    {'range': 'B4', 'values': [[spreadsheet_data['FG']['name']]]},
    {'range': 'C4', 'values': [[spreadsheet_data['BT']['name']]]},
    {'range': 'D4', 'values': [[spreadsheet_data['RT']['name']]]},
    {'range': 'E4', 'values': [[spreadsheet_data['MB']['name']]]},
    {'range': 'F4', 'values': [[spreadsheet_data['DP']['name']]]}
]
OdometersheetID.batch_update(name_updates)
print("✅ Updated all spreadsheet names in row 4")

# Generate all updates for bus numbers
all_bus_updates = []

# Helper function to create batch updates for a column
def create_column_updates(column_letter, start_row, buses):
    updates = []
    for i, bus in enumerate(buses):
        row = start_row + i
        # Try to convert numeric strings to integers
        try:
            if isinstance(bus, str) and bus.isdigit():
                bus = int(bus)
        except:
            pass
        updates.append({
            'range': f'{column_letter}{row}',
            'values': [[bus]]
        })
    return updates

# Create all updates for bus numbers
all_bus_updates.extend(create_column_updates('A', 5, spreadsheet_data['SVP']['buses']))
all_bus_updates.extend(create_column_updates('B', 5, spreadsheet_data['FG']['buses']))
all_bus_updates.extend(create_column_updates('C', 5, spreadsheet_data['BT']['buses']))
all_bus_updates.extend(create_column_updates('D', 5, spreadsheet_data['RT']['buses']))
all_bus_updates.extend(create_column_updates('E', 5, spreadsheet_data['MB']['buses']))
all_bus_updates.extend(create_column_updates('F', 5, spreadsheet_data['DP']['buses']))

# Batch update OdometersheetID with bus numbers
optimized_batch_update(OdometersheetID, all_bus_updates)
print(f"✅ Updated {len(all_bus_updates)} bus entries in OdometersheetID")

# === STEP 4: UPDATE REPORT_EO1 WORKSHEET WITH ALL BUS NUMBERS ===
print("\n📊 STEP 4: Updating Report_EO1 with all bus numbers...")

# Combine all bus lists for ReportA_EO1
all_buses_with_sources = []
for key, data in spreadsheet_data.items():
    for bus in data['buses']:
        all_buses_with_sources.append((bus, data['name']))

# Generate batch updates for Report_EO1
report_EO1_updates = []
for i, (bus, source) in enumerate(all_buses_with_sources):
    row = i + 2  # Start from row 2
    # Try to convert numeric strings to integers
    try:
        if isinstance(bus, str) and bus.isdigit():
            bus = int(bus)
    except:
        pass

    report_EO1_updates.append({
        'range': f'B{row}',
        'values': [[bus]]
    })
    report_EO1_updates.append({
        'range': f'AC{row}',
        'values': [[source]]
    })

# Batch update Report_EO1
optimized_batch_update(Report_EO1_Worksheet, report_EO1_updates)
print(f"✅ Updated {len(all_buses_with_sources)} bus entries in Report_EO1 worksheet")

print("🎉 Bus data preparation completed successfully!")

# === STEP 5: SEARCH FOR ODOMETER VALUES ===
print("\n🔍 STEP 5: Searching for odometer values...")

# Clear columns C and D
Report_EO1_Worksheet.batch_clear(["C2:D"])

# Get current date in expected format (e.g., "05,May25")
current_date_str = datetime.now().strftime('%d,%b%y')

# Get main data from Report_EO1
main_data = Report_EO1_Worksheet.get_all_values()
header, rows = main_data[0], main_data[1:]

# Map each row to bus + date (using today's date)
row_map = {}
bus_set = set()
for i, row in enumerate(rows, start=2):
    try:
        if len(row) > 1:  # Make sure row has at least 2 columns
            bus = re.sub(r'[^a-zA-Z0-9]', '', str(row[1]).strip()).upper()  # Clean bus name
            if bus:
                row_map[i] = (bus, current_date_str)
                bus_set.add(bus)
    except Exception as e:
        print(f"⚠️ Error processing row {i}: {e}")
        continue

print(f"🔎 Total buses to process: {len(bus_set)}")
if bus_set:
    print("Sample buses:", list(bus_set)[:5])  # Log first 5 buses

# Initialize lookup maps with thread safety
bus_date_value_map = {}
match_type_map = {}
fallback_date_map = {}
source_spreadsheet_map = {}
map_lock = threading.Lock()

# Function to find closest earlier date value
def find_closest_date_value(sheet_data, target_date_str, value_column_index=12):
    """Find closest earlier date with a value in the specified column"""
    if not sheet_data:
        return None, ""

    try:
        target_date = datetime.strptime(target_date_str, '%d,%b%y')
        valid_rows = []

        for row in sheet_data[1:]:  # Skip header
            if len(row) <= value_column_index:
                continue

            date_cell = row[1].strip() if len(row) > 1 else ""
            value_cell = row[value_column_index].strip() if len(row) > value_column_index else ""

            if not date_cell or not value_cell:
                continue

            # Try multiple date formats
            parsed_date = None
            for fmt in ['%d,%b%y', '%d-%b-%y', '%d/%b/%y']:
                try:
                    parsed_date = datetime.strptime(date_cell, fmt)
                    break
                except:
                    continue

            if parsed_date and parsed_date <= target_date and value_cell:
                valid_rows.append((parsed_date, date_cell, value_cell))

        if valid_rows:
            valid_rows.sort(key=lambda x: x[0], reverse=True)  # Newest date first
            best = valid_rows[0]
            return best[2], best[1]  # value, fallback_date

    except Exception as e:
        print(f"⚠️ Error in fallback for {target_date_str}: {e}")

    return None, ""

# Process a single bus to find odometer value
def process_bus_odometer(bus):
    """Process odometer data for a single bus"""
    found = False

    # Use the cached function to get worksheet data
    data, source = get_worksheet_data("All", bus)

    if data:
        found = True
        # Check for exact date match
        exact_match = False
        for row in data[1:]:  # Skip header
            if len(row) > 12 and row[1].strip() == current_date_str and row[12].strip():
                odometer_val = row[12].strip()
                if odometer_val and odometer_val.isdigit():  # Validate numeric
                    with map_lock:
                        bus_date_value_map[(bus, current_date_str)] = odometer_val
                        match_type_map[(bus, current_date_str)] = "exact"
                        fallback_date_map[(bus, current_date_str)] = current_date_str  # Store current date
                        source_spreadsheet_map[(bus, current_date_str)] = source
                    exact_match = True
                    print(f"📌 Bus {bus}: Exact match: {odometer_val} km on {current_date_str}")
                    break

        if not exact_match:
            # Fallback to closest date
            val, fallback_date = find_closest_date_value(data, current_date_str, 12)
            if val:
                with map_lock:
                    bus_date_value_map[(bus, current_date_str)] = val
                    match_type_map[(bus, current_date_str)] = "fallback"
                    fallback_date_map[(bus, current_date_str)] = fallback_date
                    source_spreadsheet_map[(bus, current_date_str)] = source
                print(f"🔄 Bus {bus}: Fallback match: {val} km on {fallback_date}")

    if not found:
        print(f"🚨 Bus {bus} not found in ANY spreadsheet!")

    return found

# Process buses in parallel batches
MAX_WORKERS = 5  # Adjust based on API limits
BATCH_SIZE = 10  # Process this many buses at once

def process_buses_in_batches(buses):
    """Process buses in parallel batches"""
    total = len(buses)
    completed = 0

    bus_chunks = chunk_list(list(buses), BATCH_SIZE)

    for chunk in bus_chunks:
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(chunk))) as executor:
            futures = {executor.submit(process_bus_odometer, bus): bus for bus in chunk}
            for future in concurrent.futures.as_completed(futures):
                bus = futures[future]
                try:
                    result = future.result()
                    completed += 1
                    if completed % 10 == 0 or completed == total:
                        print(f"Progress: {completed}/{total} buses processed ({completed/total:.1%})")
                except Exception as e:
                    print(f"⚠️ Error processing bus {bus}: {e}")

        # Small delay between batches to avoid API rate limits
        time.sleep(1)

    print(f"✅ Completed processing all {total} buses")

# Process all buses in batches
if bus_set:
    process_buses_in_batches(bus_set)

# Update Report_EO1 with logging
print("\n📝 Writing data to Report_EO1...")
updated_rows = []
for row_idx, (bus, date_str) in row_map.items():
    key = (bus, date_str)
    val = bus_date_value_map.get(key, "")
    fallback = fallback_date_map.get(key, "")
    match_type = match_type_map.get(key, "")
    source = source_spreadsheet_map.get(key, "")

    # Improved logic for Column C (date) and Column D (value)
    if match_type == "exact":
        # For exact matches, use the current date (date_str)
        col_c = date_str
        col_d = val if val else ""
    elif match_type == "fallback":
        # For fallback matches, use the fallback date we found
        col_c = fallback if fallback else ""
        col_d = val if val else ""
    else:
        # No match found
        col_c = ""
        col_d = ""

    # Add to batch update
    updated_rows.append([col_c, col_d])

# Prepare batch update
if updated_rows:
    # Split into manageable chunks
    MAX_ROWS_PER_UPDATE = 1000  # Adjust based on API limits
    row_chunks = chunk_list(updated_rows, MAX_ROWS_PER_UPDATE)

    for i, chunk in enumerate(row_chunks):
        start_row = 2 + i * MAX_ROWS_PER_UPDATE
        end_row = start_row + len(chunk) - 1
        range_to_update = f"C{start_row}:D{end_row}"

        Report_EO1_Worksheet.update(range_to_update, chunk)
        print(f"✅ Updated rows {start_row}-{end_row} in Report_EO1")

        # Small delay between batch updates
        if i < len(row_chunks) - 1:
            time.sleep(1)

    print(f"✅ All {len(updated_rows)} rows updated successfully in Report_EO1!")
else:
    print("❌ No updates made to Report_EO1.")

print("🎉 Complete script execution finished successfully!")




🚀 Initializing performance optimizations...

🧹 STEP 1: Clearing data...

📊 STEP 2: Processing bus data...
✅ Processed SVP spreadsheet: Odometer_SVP with 59 buses
✅ Processed FG spreadsheet: Odometer_FG with 20 buses
✅ Processed BT spreadsheet: Odometer_BT with 1 buses
✅ Processed RT spreadsheet: Odometer_RT with 1 buses
✅ Processed MB spreadsheet: Odometer_MB with 1 buses
✅ Processed DP spreadsheet: Odometer_DP with 1 buses

📝 STEP 3: Updating OdometersheetID with bus numbers...
✅ Updated all spreadsheet names in row 4
✅ Updated 83 bus entries in OdometersheetID

📊 STEP 4: Updating Report_EO1 with all bus numbers...
✅ Updated 83 bus entries in Report_EO1 worksheet
🎉 Bus data preparation completed successfully!

🔍 STEP 5: Searching for odometer values...
🔎 Total buses to process: 83
Sample buses: ['587', '475', '515', '486', '999']
📑 Found worksheet 587 in Odometer_SVP
🔄 Bus 587: Fallback match: 296935 km on 14,May25
📑 Found worksheet 475 in Odometer_SVP
🔄 Bus 475: Fallback match: 32721

<ipython-input-11-307866c0d17e>:447: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  Report_EO1_Worksheet.update(range_to_update, chunk)


✅ Updated rows 2-84 in Report_EO1
✅ All 83 rows updated successfully in Report_EO1!
🎉 Complete script execution finished successfully!


In [ ]:
# Here i will try to get the HSD balance on current date


from datetime import datetime
import re
import time
from collections import defaultdict
import functools
import concurrent.futures
import threading

# === PERFORMANCE OPTIMIZATION SETUP ===
print("🚀 Initializing performance optimizations...")

# Cache for worksheet data to reduce API calls
data_cache = {}
cache_lock = threading.Lock()

# LRU Cache decorator for expensive operations
def lru_cache_with_expiry(maxsize=128, typed=False, ttl=600):
    """LRU Cache with expiry time"""
    def decorator(func):
        cache_dict = {}
        cache_times = {}
        cache_lock = threading.Lock()

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = str(args) + str(kwargs)
            current_time = time.time()

            with cache_lock:
                # Check if key exists and not expired
                if key in cache_dict and current_time - cache_times[key] < ttl:
                    return cache_dict[key]

                # Call the function and cache result
                result = func(*args, **kwargs)
                cache_dict[key] = result
                cache_times[key] = current_time

                # Remove oldest entries if cache is too large
                if len(cache_dict) > maxsize:
                    oldest_key = min(cache_times, key=cache_times.get)
                    cache_dict.pop(oldest_key)
                    cache_times.pop(oldest_key)

                return result
        return wrapper
    return decorator

# Batch processing helper functions
def chunk_list(lst, chunk_size):
    """Split list into chunks of specified size"""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

# Cached function to get worksheet data
@lru_cache_with_expiry(maxsize=100, ttl=300)  # Cache for 5 minutes
def get_worksheet_data(spreadsheet_name, worksheet_name):
    """Get worksheet data with caching"""
    cache_key = f"{spreadsheet_name}:{worksheet_name}"

    with cache_lock:
        if cache_key in data_cache:
            print(f"📋 Cache hit for {cache_key}")
            return data_cache[cache_key]

    # Function to find the actual worksheet object
    def find_worksheet(spreadsheet_list, worksheet_name, spreadsheet_names):
        for idx, spreadsheet in enumerate(spreadsheet_list):
            try:
                sheet = spreadsheet.worksheet(worksheet_name)
                print(f"📑 Found worksheet {worksheet_name} in {spreadsheet_names[idx]}")
                return sheet, spreadsheet_names[idx]
            except Exception:
                continue
        return None, None

    # Cache miss, need to retrieve data
    try:
        sheet, source = find_worksheet(odometer_spreadsheets, worksheet_name, spreadsheet_names)
        if sheet:
            data = sheet.get_all_values()

            with cache_lock:
                data_cache[cache_key] = (data, source)

            return data, source
        return None, None
    except Exception as e:
        print(f"⚠️ Error retrieving data for {worksheet_name}: {e}")
        return None, None

# === STEP 1: CLEAR DATA HSD from AF Coloumn
print("\n🧹 STEP 1: Clearing data...")


clear_range_HSD_Report_EO1_Worksheet = ['AF2:AF200', 'AG2:AG200']

Report_EO1_Worksheet.batch_clear(clear_range_HSD_Report_EO1_Worksheet)


# === STEP 5: SEARCH FOR HSD Balance VALUES ===
print("\n🔍 STEP 5: Searching for HSD Balance values...")


# Get current date in expected format (e.g., "05,May25")
current_date_str = datetime.now().strftime('%d,%b%y')

# Get main data from Report_EO1
main_data = Report_EO1_Worksheet.get_all_values()
header, rows = main_data[0], main_data[1:]

# Map each row to bus + date (using today's date)
row_map = {}
bus_set = set()
for i, row in enumerate(rows, start=2):
    try:
        if len(row) > 1:  # Make sure row has at least 2 columns
            bus = re.sub(r'[^a-zA-Z0-9]', '', str(row[1]).strip()).upper()  # Clean bus name
            if bus:
                row_map[i] = (bus, current_date_str)
                bus_set.add(bus)
    except Exception as e:
        print(f"⚠️ Error processing row {i}: {e}")
        continue

print(f"🔎 Total buses to process: {len(bus_set)}")
if bus_set:
    print("Sample buses:", list(bus_set)[:5])  # Log first 5 buses

# Initialize lookup maps with thread safety
bus_date_value_map = {}
match_type_map = {}
fallback_date_map = {}
source_spreadsheet_map = {}
map_lock = threading.Lock()

# Function to find closest earlier date value
def find_closest_date_value(sheet_data, target_date_str, value_column_index=6):
    """Find closest earlier date with a value in the specified column"""
    if not sheet_data:
        return None, ""

    try:
        target_date = datetime.strptime(target_date_str, '%d,%b%y')
        valid_rows = []

        for row in sheet_data[1:]:  # Skip header
            if len(row) <= value_column_index:
                continue

            date_cell = row[1].strip() if len(row) > 1 else ""
            value_cell = row[value_column_index].strip() if len(row) > value_column_index else ""

            if not date_cell or not value_cell:
                continue

            # Try multiple date formats
            parsed_date = None
            for fmt in ['%d,%b%y', '%d-%b-%y', '%d/%b/%y']:
                try:
                    parsed_date = datetime.strptime(date_cell, fmt)
                    break
                except:
                    continue

            if parsed_date and parsed_date <= target_date and value_cell:
                valid_rows.append((parsed_date, date_cell, value_cell))

        if valid_rows:
            valid_rows.sort(key=lambda x: x[0], reverse=True)  # Newest date first
            best = valid_rows[0]
            return best[2], best[1]  # value, fallback_date

    except Exception as e:
        print(f"⚠️ Error in fallback for {target_date_str}: {e}")

    return None, ""

# Process a single bus to find odometer value
def process_bus_odometer(bus):
    """Process odometer data for a single bus"""
    found = False

    # Use the cached function to get worksheet data
    data, source = get_worksheet_data("All", bus)

    if data:
        found = True
        # Check for exact date match
        exact_match = False
        for row in data[1:]:  # Skip header
            if len(row) > 12 and row[1].strip() == current_date_str and row[12].strip():
                odometer_val = row[12].strip()
                if odometer_val and odometer_val.isdigit():  # Validate numeric
                    with map_lock:
                        bus_date_value_map[(bus, current_date_str)] = odometer_val
                        match_type_map[(bus, current_date_str)] = "exact"
                        fallback_date_map[(bus, current_date_str)] = current_date_str  # Store current date
                        source_spreadsheet_map[(bus, current_date_str)] = source
                    exact_match = True
                    print(f"📌 Bus {bus}: Exact match: {odometer_val} km on {current_date_str}")
                    break

        if not exact_match:
            # Fallback to closest date
            val, fallback_date = find_closest_date_value(data, current_date_str, 6)
            if val:
                with map_lock:
                    bus_date_value_map[(bus, current_date_str)] = val
                    match_type_map[(bus, current_date_str)] = "fallback"
                    fallback_date_map[(bus, current_date_str)] = fallback_date
                    source_spreadsheet_map[(bus, current_date_str)] = source
                print(f"🔄 Bus {bus}: Fallback match: {val} km on {fallback_date}")

    if not found:
        print(f"🚨 Bus {bus} not found in ANY spreadsheet!")

    return found

# Process buses in parallel batches
MAX_WORKERS = 5  # Adjust based on API limits
BATCH_SIZE = 10  # Process this many buses at once

def process_buses_in_batches(buses):
    """Process buses in parallel batches"""
    total = len(buses)
    completed = 0

    bus_chunks = chunk_list(list(buses), BATCH_SIZE)

    for chunk in bus_chunks:
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(chunk))) as executor:
            futures = {executor.submit(process_bus_odometer, bus): bus for bus in chunk}
            for future in concurrent.futures.as_completed(futures):
                bus = futures[future]
                try:
                    result = future.result()
                    completed += 1
                    if completed % 10 == 0 or completed == total:
                        print(f"Progress: {completed}/{total} buses processed ({completed/total:.1%})")
                except Exception as e:
                    print(f"⚠️ Error processing bus {bus}: {e}")

        # Small delay between batches to avoid API rate limits
        time.sleep(1)

    print(f"✅ Completed processing all {total} buses")

# Process all buses in batches
if bus_set:
    process_buses_in_batches(bus_set)

# Update Report_EO1 with logging
print("\n📝 Writing data to Report_EO1...")
updated_rows = []
for row_idx, (bus, date_str) in row_map.items():
    key = (bus, date_str)
    val = bus_date_value_map.get(key, "")
    fallback = fallback_date_map.get(key, "")
    match_type = match_type_map.get(key, "")
    source = source_spreadsheet_map.get(key, "")

    # Improved logic for Column C (date) and Column D (value)
    if match_type == "exact":
        # For exact matches, use the current date (date_str)
        col_c = date_str
        col_d = val if val else ""
    elif match_type == "fallback":
        # For fallback matches, use the fallback date we found
        col_c = fallback if fallback else ""
        col_d = val if val else ""
    else:
        # No match found
        col_c = ""
        col_d = ""

    # Add to batch update
    updated_rows.append([col_c, col_d])

# Prepare batch update
if updated_rows:
    # Split into manageable chunks
    MAX_ROWS_PER_UPDATE = 1000  # Adjust based on API limits
    row_chunks = chunk_list(updated_rows, MAX_ROWS_PER_UPDATE)

    for i, chunk in enumerate(row_chunks):
        start_row = 2 + i * MAX_ROWS_PER_UPDATE
        end_row = start_row + len(chunk) - 1
        range_to_update = f"AF{start_row}:AG{end_row}"

        Report_EO1_Worksheet.update(range_to_update, chunk)
        print(f"✅ Updated rows {start_row}-{end_row} in Report_EO1")

        # Small delay between batch updates
        if i < len(row_chunks) - 1:
            time.sleep(1)

    print(f"✅ All {len(updated_rows)} rows updated successfully in Report_EO1!")
else:
    print("❌ No updates made to Report_EO1.")

print("🎉 Complete script execution finished successfully!")




Based on your requirements, I've created a new script that calculates non-operating days for buses by analyzing odometer readings. The script follows your process flow and implements the specific logic needed to detect when buses are not operating.
Key Features of This Implementation:
1. Clear Existing Data

Clears columns AH, AI, and AJ in rows 2-200 of the Report_EO1_Worksheet

2. Get Current Date

Gets the current date and formats it as DD,MonYY
Calculates reference dates for different time periods (week, month, year)

3. Process Buses

For each bus, the script:

Retrieves all available date and odometer data
Sorts the data chronologically
Identifies non-operating days through two methods:

When odometer readings don't change between consecutive dates (indicating the bus didn't run)
When there are gaps between dates (missing days)


Counts non-operating days in three time periods:

Last week
Last month (30 days)
Last year (365 days)





4. Update Report

Updates columns AH, AI, and AJ with the calculated non-operating days
Uses batch processing to efficiently update the worksheet

How the Non-Operating Days Detection Works
The script identifies non-operating days in two ways:

Same odometer value on consecutive dates: When the odometer reading doesn't change between two dates, it means the bus didn't operate on the later date.
Missing dates: When there's a gap between two dates in the data, the script assumes the bus might not have operated on those missing days.

For example, with the data you provided:
03,Jan24 - 488487
05,Jan24 - 488815
06,Jan24 - 488815
08,Jan24 - 489011
09,Jan24 - 489339
10,Jan24 - 489643
The script would detect:

Non-operating on 04,Jan24 (missing date)
Non-operating on 06,Jan24 (same odometer as 05,Jan24)
Non-operating on 07,Jan24 (missing date)

Would you like me to explain any specific part of the implementation in more detail?

In [ ]:
# No of days bus operated in last One week, One Month and One year

from datetime import datetime, timedelta
import re
import time
from collections import defaultdict
import functools
import concurrent.futures
import threading

# === PERFORMANCE OPTIMIZATION SETUP ===
print("🚀 Initializing performance optimizations...")

# Cache for worksheet data to reduce API calls
data_cache = {}
cache_lock = threading.Lock()

# LRU Cache decorator for expensive operations
def lru_cache_with_expiry(maxsize=128, typed=False, ttl=600):
    """LRU Cache with expiry time"""
    def decorator(func):
        cache_dict = {}
        cache_times = {}
        cache_lock = threading.Lock()

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = str(args) + str(kwargs)
            current_time = time.time()

            with cache_lock:
                # Check if key exists and not expired
                if key in cache_dict and current_time - cache_times[key] < ttl:
                    return cache_dict[key]

                # Call the function and cache result
                result = func(*args, **kwargs)
                cache_dict[key] = result
                cache_times[key] = current_time

                # Remove oldest entries if cache is too large
                if len(cache_dict) > maxsize:
                    oldest_key = min(cache_times, key=cache_times.get)
                    cache_dict.pop(oldest_key)
                    cache_times.pop(oldest_key)

                return result
        return wrapper
    return decorator

# Batch processing helper functions
def chunk_list(lst, chunk_size):
    """Split list into chunks of specified size"""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

# Cached function to get worksheet data
@lru_cache_with_expiry(maxsize=100, ttl=300)  # Cache for 5 minutes
def get_worksheet_data(spreadsheet_name, worksheet_name):
    """Get worksheet data with caching"""
    cache_key = f"{spreadsheet_name}:{worksheet_name}"

    with cache_lock:
        if cache_key in data_cache:
            print(f"📋 Cache hit for {cache_key}")
            return data_cache[cache_key]

    # Function to find the actual worksheet object
    def find_worksheet(spreadsheet_list, worksheet_name, spreadsheet_names):
        for idx, spreadsheet in enumerate(spreadsheet_list):
            try:
                sheet = spreadsheet.worksheet(worksheet_name)
                print(f"📑 Found worksheet {worksheet_name} in {spreadsheet_names[idx]}")
                return sheet, spreadsheet_names[idx]
            except Exception:
                continue
        return None, None

    # Cache miss, need to retrieve data
    try:
        sheet, source = find_worksheet(odometer_spreadsheets, worksheet_name, spreadsheet_names)
        if sheet:
            data = sheet.get_all_values()

            with cache_lock:
                data_cache[cache_key] = (data, source)

            return data, source
        return None, None
    except Exception as e:
        print(f"⚠️ Error retrieving data for {worksheet_name}: {e}")
        return None, None

# === STEP 1: CLEAR DATA from AH, AI, AJ Columns ===
print("\n🧹 STEP 1: Clearing data...")

clear_range_Report_EO1_Worksheet = ['AH2:AJ200']
Report_EO1_Worksheet.batch_clear(clear_range_Report_EO1_Worksheet)

# === STEP 2: GET CURRENT DATE ===
print("\n📅 STEP 2: Getting current date...")
current_date = datetime.now()
current_date_str = current_date.strftime('%d,%b%y')

# Calculate dates for different time periods
one_week_ago = current_date - timedelta(days=7)
one_month_ago = current_date - timedelta(days=30)
one_year_ago = current_date - timedelta(days=365)

# === STEP 3: PROCESS BUSES ===
print("\n🚌 STEP 3: Processing buses...")

# Get main data from Report_EO1
main_data = Report_EO1_Worksheet.get_all_values()
header, rows = main_data[0], main_data[1:]

# Map each row to bus
row_map = {}
bus_set = set()
for i, row in enumerate(rows, start=2):
    try:
        if len(row) > 1:  # Make sure row has at least 2 columns
            bus = re.sub(r'[^a-zA-Z0-9]', '', str(row[1]).strip()).upper()  # Clean bus name
            if bus:
                row_map[i] = bus
                bus_set.add(bus)
    except Exception as e:
        print(f"⚠️ Error processing row {i}: {e}")
        continue

print(f"🔎 Total buses to process: {len(bus_set)}")
if bus_set:
    print("Sample buses:", list(bus_set)[:5])  # Log first 5 buses

# Initialize results with thread safety
bus_results = {}
results_lock = threading.Lock()

def parse_date(date_str):
    """Parse date string with multiple format support"""
    for fmt in ['%d,%b%y', '%d-%b-%y', '%d/%b/%y']:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            continue
    return None

def calculate_non_operating_days(bus):
    """Calculate non-operating days for a bus across different time periods"""
    try:
        # Get the worksheet data for this bus
        data, source = get_worksheet_data("All", bus)

        if not data or len(data) < 2:  # Need at least header + 1 data row
            print(f"⚠️ No data found for bus {bus}")
            return None

        # Extract date and odometer data
        # Assuming column B (index 1) is date and column M (index 12) is odometer
        date_odometer_data = []

        for row in data[1:]:  # Skip header
            if len(row) <= 12:  # Skip rows that don't have enough columns
                continue

            date_str = row[1].strip()
            odometer_str = row[12].strip()

            if not date_str or not odometer_str:
                continue

            parsed_date = parse_date(date_str)
            if not parsed_date:
                continue

            try:
                odometer_value = int(odometer_str)
                date_odometer_data.append((parsed_date, odometer_value))
            except ValueError:
                continue

        if not date_odometer_data:
            print(f"⚠️ No valid date-odometer pairs found for bus {bus}")
            return None

        # Sort by date in ascending order
        date_odometer_data.sort(key=lambda x: x[0])

        # Identify non-operating days
        non_operating_days = []

        for i in range(1, len(date_odometer_data)):
            current_date, current_odo = date_odometer_data[i]
            prev_date, prev_odo = date_odometer_data[i-1]

            # If odometer didn't change, the bus didn't operate
            if current_odo == prev_odo:
                non_operating_days.append(current_date)
                continue

            # Check for missing days between consecutive dates
            date_diff = (current_date - prev_date).days
            if date_diff > 1:
                # For all days in between, determine if they were likely non-operating
                for day_offset in range(1, date_diff):
                    missing_date = prev_date + timedelta(days=day_offset)
                    non_operating_days.append(missing_date)

        # Count non-operating days in different periods
        week_non_op = 0
        month_non_op = 0
        year_non_op = 0

        for non_op_date in non_operating_days:
            if non_op_date >= one_week_ago:
                week_non_op += 1
            if non_op_date >= one_month_ago:
                month_non_op += 1
            if non_op_date >= one_year_ago:
                year_non_op += 1

        # Calculate results
        with results_lock:
            bus_results[bus] = {
                'week_non_op': week_non_op,
                'month_non_op': month_non_op,
                'year_non_op': year_non_op
            }

        print(f"✅ Bus {bus}: Week({week_non_op}), Month({month_non_op}), Year({year_non_op}) non-operating days")
        return True

    except Exception as e:
        print(f"❌ Error processing bus {bus}: {e}")
        return None

# Process buses in parallel batches
MAX_WORKERS = 5  # Adjust based on API limits
BATCH_SIZE = 10  # Process this many buses at once

def process_buses_in_batches(buses):
    """Process buses in parallel batches"""
    total = len(buses)
    completed = 0

    bus_chunks = chunk_list(list(buses), BATCH_SIZE)

    for chunk in bus_chunks:
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(chunk))) as executor:
            futures = {executor.submit(calculate_non_operating_days, bus): bus for bus in chunk}
            for future in concurrent.futures.as_completed(futures):
                bus = futures[future]
                try:
                    result = future.result()
                    completed += 1
                    if completed % 10 == 0 or completed == total:
                        print(f"Progress: {completed}/{total} buses processed ({completed/total:.1%})")
                except Exception as e:
                    print(f"⚠️ Error processing bus {bus}: {e}")

        # Small delay between batches to avoid API rate limits
        time.sleep(1)

    print(f"✅ Completed processing all {total} buses")

# Process all buses in batches
if bus_set:
    process_buses_in_batches(bus_set)

# === STEP 4: UPDATE REPORT ===
print("\n📝 STEP 4: Updating report...")

# Prepare data for batch update
updated_rows = []
for row_idx, bus in row_map.items():
    result = bus_results.get(bus, {})

    week_non_op = result.get('week_non_op', '')
    month_non_op = result.get('month_non_op', '')
    year_non_op = result.get('year_non_op', '')

    updated_rows.append([week_non_op, month_non_op, year_non_op])

# Prepare batch update
if updated_rows:
    # Split into manageable chunks
    MAX_ROWS_PER_UPDATE = 1000  # Adjust based on API limits
    row_chunks = chunk_list(updated_rows, MAX_ROWS_PER_UPDATE)

    for i, chunk in enumerate(row_chunks):
        start_row = 2 + i * MAX_ROWS_PER_UPDATE
        end_row = start_row + len(chunk) - 1
        range_to_update = f"AH{start_row}:AJ{end_row}"

        Report_EO1_Worksheet.update(range_to_update, chunk)
        print(f"✅ Updated rows {start_row}-{end_row} in Report_EO1")

        # Small delay between batch updates
        if i < len(row_chunks) - 1:
            time.sleep(1)

    print(f"✅ All {len(updated_rows)} rows updated successfully in Report_EO1!")
else:
    print("❌ No updates made to Report_EO1.")

print("🎉 Process finished successfully!")


In [14]:
#R2

from datetime import datetime, timedelta
import re
import time
from collections import defaultdict
import functools
import concurrent.futures
import threading

# === PERFORMANCE OPTIMIZATION SETUP ===
print("🚀 Initializing performance optimizations...")

# Cache for worksheet data to reduce API calls
data_cache = {}
cache_lock = threading.Lock()

# LRU Cache decorator for expensive operations
def lru_cache_with_expiry(maxsize=128, typed=False, ttl=600):
    """LRU Cache with expiry time"""
    def decorator(func):
        cache_dict = {}
        cache_times = {}
        cache_lock = threading.Lock()

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = str(args) + str(kwargs)
            current_time = time.time()

            with cache_lock:
                # Check if key exists and not expired
                if key in cache_dict and current_time - cache_times[key] < ttl:
                    return cache_dict[key]

                # Call the function and cache result
                result = func(*args, **kwargs)
                cache_dict[key] = result
                cache_times[key] = current_time

                # Remove oldest entries if cache is too large
                if len(cache_dict) > maxsize:
                    oldest_key = min(cache_times, key=cache_times.get)
                    cache_dict.pop(oldest_key)
                    cache_times.pop(oldest_key)

                return result
        return wrapper
    return decorator

# Batch processing helper functions
def chunk_list(lst, chunk_size):
    """Split list into chunks of specified size"""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

# Cached function to get worksheet data
@lru_cache_with_expiry(maxsize=100, ttl=300)  # Cache for 5 minutes
def get_worksheet_data(spreadsheet_name, worksheet_name):
    """Get worksheet data with caching"""
    cache_key = f"{spreadsheet_name}:{worksheet_name}"

    with cache_lock:
        if cache_key in data_cache:
            print(f"📋 Cache hit for {cache_key}")
            return data_cache[cache_key]

    # Function to find the actual worksheet object
    def find_worksheet(spreadsheet_list, worksheet_name, spreadsheet_names):
        for idx, spreadsheet in enumerate(spreadsheet_list):
            try:
                sheet = spreadsheet.worksheet(worksheet_name)
                print(f"📑 Found worksheet {worksheet_name} in {spreadsheet_names[idx]}")
                return sheet, spreadsheet_names[idx]
            except Exception:
                continue
        return None, None

    # Cache miss, need to retrieve data
    try:
        sheet, source = find_worksheet(odometer_spreadsheets, worksheet_name, spreadsheet_names)
        if sheet:
            data = sheet.get_all_values()

            with cache_lock:
                data_cache[cache_key] = (data, source)

            return data, source
        return None, None
    except Exception as e:
        print(f"⚠️ Error retrieving data for {worksheet_name}: {e}")
        return None, None

# === STEP 1: CLEAR DATA from AH, AI, AJ Columns ===
print("\n🧹 STEP 1: Clearing data...")

clear_range_Report_EO1_Worksheet = ['AH2:AJ200']
Report_EO1_Worksheet.batch_clear(clear_range_Report_EO1_Worksheet)

# === STEP 2: GET CURRENT DATE ===
print("\n📅 STEP 2: Getting current date...")
current_date = datetime.now()
current_date_str = current_date.strftime('%d,%b%y')

# Calculate dates for different time periods
one_week_ago = current_date - timedelta(days=7)
one_month_ago = current_date - timedelta(days=30)
one_year_ago = current_date - timedelta(days=365)

# === STEP 3: PROCESS BUSES ===
print("\n🚌 STEP 3: Processing buses...")

# Get main data from Report_EO1
main_data = Report_EO1_Worksheet.get_all_values()
header, rows = main_data[0], main_data[1:]

# Map each row to bus
row_map = {}
bus_set = set()
for i, row in enumerate(rows, start=2):
    try:
        if len(row) > 1:  # Make sure row has at least 2 columns
            bus = re.sub(r'[^a-zA-Z0-9]', '', str(row[1]).strip()).upper()  # Clean bus name
            if bus:
                row_map[i] = bus
                bus_set.add(bus)
    except Exception as e:
        print(f"⚠️ Error processing row {i}: {e}")
        continue

print(f"🔎 Total buses to process: {len(bus_set)}")
if bus_set:
    print("Sample buses:", list(bus_set)[:5])  # Log first 5 buses

# Initialize results with thread safety
bus_results = {}
results_lock = threading.Lock()

def parse_date(date_str):
    """Parse date string with multiple format support"""
    for fmt in ['%d,%b%y', '%d-%b-%y', '%d/%b/%y']:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            continue
    return None

def calculate_non_operating_days(bus):
    """Calculate non-operating days for a bus across different time periods"""
    try:
        # Get the worksheet data for this bus
        data, source = get_worksheet_data("All", bus)

        if not data or len(data) < 2:  # Need at least header + 1 data row
            print(f"⚠️ No data found for bus {bus}")
            return None

        # Extract date and odometer data
        # Column B (index 1) is date and column M (index 12) is odometer
        date_odometer_map = {}

        for row in data[1:]:  # Skip header
            if len(row) <= 12:  # Skip rows that don't have enough columns
                continue

            date_str = row[1].strip()
            odometer_str = row[12].strip()

            if not date_str or not odometer_str:
                continue

            parsed_date = parse_date(date_str)
            if not parsed_date:
                continue

            try:
                odometer_value = int(odometer_str)
                # Use date as key (without time) to group multiple entries on same date
                date_key = parsed_date.date()

                # For multiple entries on same date, keep the highest odometer value
                if date_key in date_odometer_map:
                    date_odometer_map[date_key] = max(date_odometer_map[date_key], odometer_value)
                else:
                    date_odometer_map[date_key] = odometer_value

            except ValueError:
                continue

        if not date_odometer_map:
            print(f"⚠️ No valid date-odometer pairs found for bus {bus}")
            return None

        # Sort unique dates in ascending order
        sorted_dates = sorted(date_odometer_map.keys())

        # Identify non-operating days (only between dates, not on the same date)
        non_operating_days = []

        # Find date ranges where bus was not operating
        for i in range(1, len(sorted_dates)):
            current_date = sorted_dates[i]
            prev_date = sorted_dates[i-1]

            # Get odometer values
            current_odo = date_odometer_map[current_date]
            prev_odo = date_odometer_map[prev_date]

            # If odometer value didn't change between different dates, bus didn't operate on current date
            if current_odo == prev_odo and current_date != prev_date:
                non_operating_days.append(current_date)

            # Check for missing days between consecutive dates
            date_diff = (current_date - prev_date).days
            if date_diff > 1:
                # For all days in between, mark as non-operating
                for day_offset in range(1, date_diff):
                    missing_date = prev_date + timedelta(days=day_offset)
                    non_operating_days.append(missing_date.date())

        # Count non-operating days in different periods
        week_non_op = 0
        month_non_op = 0
        year_non_op = 0

        for non_op_date in non_operating_days:
            # Convert to datetime for comparison with time periods
            non_op_datetime = datetime.combine(non_op_date, datetime.min.time())

            if non_op_datetime >= one_week_ago:
                week_non_op += 1
            if non_op_datetime >= one_month_ago:
                month_non_op += 1
            if non_op_datetime >= one_year_ago:
                year_non_op += 1

        # Calculate results
        with results_lock:
            bus_results[bus] = {
                'week_non_op': week_non_op,
                'month_non_op': month_non_op,
                'year_non_op': year_non_op
            }

        print(f"✅ Bus {bus}: Week({week_non_op}), Month({month_non_op}), Year({year_non_op}) non-operating days")
        return True

    except Exception as e:
        print(f"❌ Error processing bus {bus}: {e}")
        return None

# Process buses in parallel batches
MAX_WORKERS = 5  # Adjust based on API limits
BATCH_SIZE = 10  # Process this many buses at once

def process_buses_in_batches(buses):
    """Process buses in parallel batches"""
    total = len(buses)
    completed = 0

    bus_chunks = chunk_list(list(buses), BATCH_SIZE)

    for chunk in bus_chunks:
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(chunk))) as executor:
            futures = {executor.submit(calculate_non_operating_days, bus): bus for bus in chunk}
            for future in concurrent.futures.as_completed(futures):
                bus = futures[future]
                try:
                    result = future.result()
                    completed += 1
                    if completed % 10 == 0 or completed == total:
                        print(f"Progress: {completed}/{total} buses processed ({completed/total:.1%})")
                except Exception as e:
                    print(f"⚠️ Error processing bus {bus}: {e}")

        # Small delay between batches to avoid API rate limits
        time.sleep(1)

    print(f"✅ Completed processing all {total} buses")

# Process all buses in batches
if bus_set:
    process_buses_in_batches(bus_set)

# === STEP 4: UPDATE REPORT ===
print("\n📝 STEP 4: Updating report...")

# Prepare data for batch update
updated_rows = []
for row_idx, bus in row_map.items():
    result = bus_results.get(bus, {})

    week_non_op = result.get('week_non_op', '')
    month_non_op = result.get('month_non_op', '')
    year_non_op = result.get('year_non_op', '')

    updated_rows.append([week_non_op, month_non_op, year_non_op])

# Prepare batch update
if updated_rows:
    # Split into manageable chunks
    MAX_ROWS_PER_UPDATE = 1000  # Adjust based on API limits
    row_chunks = chunk_list(updated_rows, MAX_ROWS_PER_UPDATE)

    for i, chunk in enumerate(row_chunks):
        start_row = 2 + i * MAX_ROWS_PER_UPDATE
        end_row = start_row + len(chunk) - 1
        range_to_update = f"AH{start_row}:AJ{end_row}"

        Report_EO1_Worksheet.update(range_to_update, chunk)
        print(f"✅ Updated rows {start_row}-{end_row} in Report_EO1")

        # Small delay between batch updates
        if i < len(row_chunks) - 1:
            time.sleep(1)

    print(f"✅ All {len(updated_rows)} rows updated successfully in Report_EO1!")
else:
    print("❌ No updates made to Report_EO1.")

print("🎉 Process finished successfully!")

🚀 Initializing performance optimizations...

🧹 STEP 1: Clearing data...

📅 STEP 2: Getting current date...

🚌 STEP 3: Processing buses...
🔎 Total buses to process: 83
Sample buses: ['587', '475', '515', '486', '999']
📑 Found worksheet 587 in Odometer_SVP
❌ Error processing bus 587: 'datetime.date' object has no attribute 'date'
📑 Found worksheet 475 in Odometer_SVP
❌ Error processing bus 475: 'datetime.date' object has no attribute 'date'
📑 Found worksheet 515 in Odometer_SVP
❌ Error processing bus 515: 'datetime.date' object has no attribute 'date'
📑 Found worksheet 486 in Odometer_SVP
❌ Error processing bus 486: 'datetime.date' object has no attribute 'date'
📑 Found worksheet 999 in Odometer_BT
❌ Error processing bus 999: 'datetime.date' object has no attribute 'date'
📑 Found worksheet 578 in Odometer_SVP
❌ Error processing bus 578: 'datetime.date' object has no attribute 'date'
📑 Found worksheet 440 in Odometer_SVP
❌ Error processing bus 440: 'datetime.date' object has no attribute 

<ipython-input-14-ea82b1898bef>:309: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  Report_EO1_Worksheet.update(range_to_update, chunk)


In [15]:
#R3

from datetime import datetime, timedelta
import re
import time
from collections import defaultdict
import functools
import concurrent.futures
import threading

# === PERFORMANCE OPTIMIZATION SETUP ===
print("🚀 Initializing performance optimizations...")

# Cache for worksheet data to reduce API calls
data_cache = {}
cache_lock = threading.Lock()

# LRU Cache decorator for expensive operations
def lru_cache_with_expiry(maxsize=128, typed=False, ttl=600):
    """LRU Cache with expiry time"""
    def decorator(func):
        cache_dict = {}
        cache_times = {}
        cache_lock = threading.Lock()

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = str(args) + str(kwargs)
            current_time = time.time()

            with cache_lock:
                # Check if key exists and not expired
                if key in cache_dict and current_time - cache_times[key] < ttl:
                    return cache_dict[key]

                # Call the function and cache result
                result = func(*args, **kwargs)
                cache_dict[key] = result
                cache_times[key] = current_time

                # Remove oldest entries if cache is too large
                if len(cache_dict) > maxsize:
                    oldest_key = min(cache_times, key=cache_times.get)
                    cache_dict.pop(oldest_key)
                    cache_times.pop(oldest_key)

                return result
        return wrapper
    return decorator

# Batch processing helper functions
def chunk_list(lst, chunk_size):
    """Split list into chunks of specified size"""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

# Cached function to get worksheet data
@lru_cache_with_expiry(maxsize=100, ttl=300)  # Cache for 5 minutes
def get_worksheet_data(spreadsheet_name, worksheet_name):
    """Get worksheet data with caching"""
    cache_key = f"{spreadsheet_name}:{worksheet_name}"

    with cache_lock:
        if cache_key in data_cache:
            print(f"📋 Cache hit for {cache_key}")
            return data_cache[cache_key]

    # Function to find the actual worksheet object
    def find_worksheet(spreadsheet_list, worksheet_name, spreadsheet_names):
        for idx, spreadsheet in enumerate(spreadsheet_list):
            try:
                sheet = spreadsheet.worksheet(worksheet_name)
                print(f"📑 Found worksheet {worksheet_name} in {spreadsheet_names[idx]}")
                return sheet, spreadsheet_names[idx]
            except Exception:
                continue
        return None, None

    # Cache miss, need to retrieve data
    try:
        sheet, source = find_worksheet(odometer_spreadsheets, worksheet_name, spreadsheet_names)
        if sheet:
            data = sheet.get_all_values()

            with cache_lock:
                data_cache[cache_key] = (data, source)

            return data, source
        return None, None
    except Exception as e:
        print(f"⚠️ Error retrieving data for {worksheet_name}: {e}")
        return None, None

# === STEP 1: CLEAR DATA from AH, AI, AJ Columns ===
print("\n🧹 STEP 1: Clearing data...")

clear_range_Report_EO1_Worksheet = ['AH2:AJ200']
Report_EO1_Worksheet.batch_clear(clear_range_Report_EO1_Worksheet)

# === STEP 2: GET CURRENT DATE ===
print("\n📅 STEP 2: Getting current date...")
current_date = datetime.now()
current_date_str = current_date.strftime('%d,%b%y')

# Calculate dates for different time periods
one_week_ago = current_date - timedelta(days=7)
one_month_ago = current_date - timedelta(days=30)
one_year_ago = current_date - timedelta(days=365)

# === STEP 3: PROCESS BUSES ===
print("\n🚌 STEP 3: Processing buses...")

# Get main data from Report_EO1
main_data = Report_EO1_Worksheet.get_all_values()
header, rows = main_data[0], main_data[1:]

# Map each row to bus
row_map = {}
bus_set = set()
for i, row in enumerate(rows, start=2):
    try:
        if len(row) > 1:  # Make sure row has at least 2 columns
            bus = re.sub(r'[^a-zA-Z0-9]', '', str(row[1]).strip()).upper()  # Clean bus name
            if bus:
                row_map[i] = bus
                bus_set.add(bus)
    except Exception as e:
        print(f"⚠️ Error processing row {i}: {e}")
        continue

print(f"🔎 Total buses to process: {len(bus_set)}")
if bus_set:
    print("Sample buses:", list(bus_set)[:5])  # Log first 5 buses

# Initialize results with thread safety
bus_results = {}
results_lock = threading.Lock()

def parse_date(date_str):
    """Parse date string with multiple format support"""
    for fmt in ['%d,%b%y', '%d-%b-%y', '%d/%b/%y']:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            continue
    return None

def calculate_non_operating_days(bus):
    """Calculate non-operating days for a bus across different time periods"""
    try:
        # Get the worksheet data for this bus
        data, source = get_worksheet_data("All", bus)

        if not data or len(data) < 2:  # Need at least header + 1 data row
            print(f"⚠️ No data found for bus {bus}")
            return None

        # Extract date and odometer data
        # Column B (index 1) is date and column M (index 12) is odometer
        date_odometer_map = {}

        for row in data[1:]:  # Skip header
            if len(row) <= 12:  # Skip rows that don't have enough columns
                continue

            date_str = row[1].strip()
            odometer_str = row[12].strip()

            if not date_str or not odometer_str:
                continue

            parsed_date = parse_date(date_str)
            if not parsed_date:
                continue

            try:
                odometer_value = int(odometer_str)
                # Use date as key (without time) to group multiple entries on same date
                date_key = parsed_date.date()

                # For multiple entries on same date, keep the highest odometer value
                if date_key in date_odometer_map:
                    date_odometer_map[date_key] = max(date_odometer_map[date_key], odometer_value)
                else:
                    date_odometer_map[date_key] = odometer_value

            except ValueError:
                continue

        if not date_odometer_map:
            print(f"⚠️ No valid date-odometer pairs found for bus {bus}")
            return None

        # Sort unique dates in ascending order
        sorted_dates = sorted(date_odometer_map.keys())

        # Identify non-operating days (only between dates, not on the same date)
        non_operating_days = []

        # Find date ranges where bus was not operating
        for i in range(1, len(sorted_dates)):
            current_date = sorted_dates[i]
            prev_date = sorted_dates[i-1]

            # Get odometer values
            current_odo = date_odometer_map[current_date]
            prev_odo = date_odometer_map[prev_date]

            # If odometer value didn't change between different dates, bus didn't operate on current date
            if current_odo == prev_odo and current_date != prev_date:
                non_operating_days.append(current_date)

            # Check for missing days between consecutive dates
            date_diff = (current_date - prev_date).days
            if date_diff > 1:
                # For all days in between, mark as non-operating
                for day_offset in range(1, date_diff):
                    missing_date = prev_date + timedelta(days=day_offset)
                    non_operating_days.append(missing_date)

        # Count non-operating days in different periods
        week_non_op = 0
        month_non_op = 0
        year_non_op = 0

        for non_op_date in non_operating_days:
            # Convert to datetime for comparison with time periods
            non_op_datetime = datetime.combine(non_op_date, datetime.min.time())

            if non_op_datetime >= one_week_ago:
                week_non_op += 1
            if non_op_datetime >= one_month_ago:
                month_non_op += 1
            if non_op_datetime >= one_year_ago:
                year_non_op += 1

        # Calculate results
        with results_lock:
            bus_results[bus] = {
                'week_non_op': week_non_op,
                'month_non_op': month_non_op,
                'year_non_op': year_non_op
            }

        print(f"✅ Bus {bus}: Week({week_non_op}), Month({month_non_op}), Year({year_non_op}) non-operating days")
        return True

    except Exception as e:
        print(f"❌ Error processing bus {bus}: {e}")
        return None

# Process buses in parallel batches
MAX_WORKERS = 5  # Adjust based on API limits
BATCH_SIZE = 10  # Process this many buses at once

def process_buses_in_batches(buses):
    """Process buses in parallel batches"""
    total = len(buses)
    completed = 0

    bus_chunks = chunk_list(list(buses), BATCH_SIZE)

    for chunk in bus_chunks:
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(chunk))) as executor:
            futures = {executor.submit(calculate_non_operating_days, bus): bus for bus in chunk}
            for future in concurrent.futures.as_completed(futures):
                bus = futures[future]
                try:
                    result = future.result()
                    completed += 1
                    if completed % 10 == 0 or completed == total:
                        print(f"Progress: {completed}/{total} buses processed ({completed/total:.1%})")
                except Exception as e:
                    print(f"⚠️ Error processing bus {bus}: {e}")

        # Small delay between batches to avoid API rate limits
        time.sleep(1)

    print(f"✅ Completed processing all {total} buses")

# Process all buses in batches
if bus_set:
    process_buses_in_batches(bus_set)

# === STEP 4: UPDATE REPORT ===
print("\n📝 STEP 4: Updating report...")

# Prepare data for batch update
updated_rows = []
for row_idx, bus in row_map.items():
    result = bus_results.get(bus, {})

    week_non_op = result.get('week_non_op', '')
    month_non_op = result.get('month_non_op', '')
    year_non_op = result.get('year_non_op', '')

    updated_rows.append([week_non_op, month_non_op, year_non_op])

# Prepare batch update
if updated_rows:
    # Split into manageable chunks
    MAX_ROWS_PER_UPDATE = 1000  # Adjust based on API limits
    row_chunks = chunk_list(updated_rows, MAX_ROWS_PER_UPDATE)

    for i, chunk in enumerate(row_chunks):
        start_row = 2 + i * MAX_ROWS_PER_UPDATE
        end_row = start_row + len(chunk) - 1
        range_to_update = f"AH{start_row}:AJ{end_row}"

        Report_EO1_Worksheet.update(range_to_update, chunk)
        print(f"✅ Updated rows {start_row}-{end_row} in Report_EO1")

        # Small delay between batch updates
        if i < len(row_chunks) - 1:
            time.sleep(1)

    print(f"✅ All {len(updated_rows)} rows updated successfully in Report_EO1!")
else:
    print("❌ No updates made to Report_EO1.")

print("🎉 Process finished successfully!")

🚀 Initializing performance optimizations...

🧹 STEP 1: Clearing data...

📅 STEP 2: Getting current date...

🚌 STEP 3: Processing buses...
🔎 Total buses to process: 83
Sample buses: ['587', '475', '515', '486', '999']
📑 Found worksheet 587 in Odometer_SVP
✅ Bus 587: Week(122), Month(131), Year(349) non-operating days
📑 Found worksheet 475 in Odometer_SVP
✅ Bus 475: Week(0), Month(0), Year(260) non-operating days
📑 Found worksheet 515 in Odometer_SVP
✅ Bus 515: Week(0), Month(0), Year(94) non-operating days
📑 Found worksheet 486 in Odometer_SVP
✅ Bus 486: Week(0), Month(0), Year(245) non-operating days
📑 Found worksheet 999 in Odometer_BT
✅ Bus 999: Week(0), Month(0), Year(78) non-operating days
📑 Found worksheet 578 in Odometer_SVP
✅ Bus 578: Week(3), Month(14), Year(220) non-operating days
📑 Found worksheet 440 in Odometer_SVP
✅ Bus 440: Week(205), Month(224), Year(398) non-operating days
📑 Found worksheet 508 in Odometer_SVP
✅ Bus 508: Week(614), Month(623), Year(814) non-operating da

<ipython-input-15-6a3cc092d4d0>:309: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  Report_EO1_Worksheet.update(range_to_update, chunk)


In [16]:
#R5

from datetime import datetime, timedelta
import re
import time
from collections import defaultdict
import functools
import concurrent.futures
import threading

# === PERFORMANCE OPTIMIZATION SETUP ===
print("🚀 Initializing performance optimizations...")

# Cache for worksheet data to reduce API calls
data_cache = {}
cache_lock = threading.Lock()

# LRU Cache decorator for expensive operations
def lru_cache_with_expiry(maxsize=128, typed=False, ttl=600):
    """LRU Cache with expiry time"""
    def decorator(func):
        cache_dict = {}
        cache_times = {}
        cache_lock = threading.Lock()

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = str(args) + str(kwargs)
            current_time = time.time()

            with cache_lock:
                # Check if key exists and not expired
                if key in cache_dict and current_time - cache_times[key] < ttl:
                    return cache_dict[key]

                # Call the function and cache result
                result = func(*args, **kwargs)
                cache_dict[key] = result
                cache_times[key] = current_time

                # Remove oldest entries if cache is too large
                if len(cache_dict) > maxsize:
                    oldest_key = min(cache_times, key=cache_times.get)
                    cache_dict.pop(oldest_key)
                    cache_times.pop(oldest_key)

                return result
        return wrapper
    return decorator

# Batch processing helper functions
def chunk_list(lst, chunk_size):
    """Split list into chunks of specified size"""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

# Cached function to get worksheet data
@lru_cache_with_expiry(maxsize=100, ttl=300)  # Cache for 5 minutes
def get_worksheet_data(spreadsheet_name, worksheet_name):
    """Get worksheet data with caching"""
    cache_key = f"{spreadsheet_name}:{worksheet_name}"

    with cache_lock:
        if cache_key in data_cache:
            print(f"📋 Cache hit for {cache_key}")
            return data_cache[cache_key]

    # Function to find the actual worksheet object
    def find_worksheet(spreadsheet_list, worksheet_name, spreadsheet_names):
        for idx, spreadsheet in enumerate(spreadsheet_list):
            try:
                sheet = spreadsheet.worksheet(worksheet_name)
                print(f"📑 Found worksheet {worksheet_name} in {spreadsheet_names[idx]}")
                return sheet, spreadsheet_names[idx]
            except Exception:
                continue
        return None, None

    # Cache miss, need to retrieve data
    try:
        sheet, source = find_worksheet(odometer_spreadsheets, worksheet_name, spreadsheet_names)
        if sheet:
            data = sheet.get_all_values()

            with cache_lock:
                data_cache[cache_key] = (data, source)

            return data, source
        return None, None
    except Exception as e:
        print(f"⚠️ Error retrieving data for {worksheet_name}: {e}")
        return None, None

# === STEP 1: CLEAR DATA from AH, AI, AJ Columns ===
print("\n🧹 STEP 1: Clearing data...")

clear_range_Report_EO1_Worksheet = ['AH2:AJ200']
Report_EO1_Worksheet.batch_clear(clear_range_Report_EO1_Worksheet)

# === STEP 2: GET CURRENT DATE ===
print("\n📅 STEP 2: Getting current date...")
current_date = datetime.now()
current_date_str = current_date.strftime('%d,%b%y')

# Calculate dates for different time periods
one_week_ago = current_date - timedelta(days=7)
one_month_ago = current_date - timedelta(days=30)
one_year_ago = current_date - timedelta(days=365)

# === STEP 3: PROCESS BUSES ===
print("\n🚌 STEP 3: Processing buses...")

# Get main data from Report_EO1
main_data = Report_EO1_Worksheet.get_all_values()
header, rows = main_data[0], main_data[1:]

# Map each row to bus
row_map = {}
bus_set = set()
for i, row in enumerate(rows, start=2):
    try:
        if len(row) > 1:  # Make sure row has at least 2 columns
            bus = re.sub(r'[^a-zA-Z0-9]', '', str(row[1]).strip()).upper()  # Clean bus name
            if bus:
                row_map[i] = bus
                bus_set.add(bus)
    except Exception as e:
        print(f"⚠️ Error processing row {i}: {e}")
        continue

print(f"🔎 Total buses to process: {len(bus_set)}")
if bus_set:
    print("Sample buses:", list(bus_set)[:5])  # Log first 5 buses

# Initialize results with thread safety
bus_results = {}
results_lock = threading.Lock()

def parse_date(date_str):
    """Parse date string with multiple format support"""
    for fmt in ['%d,%b%y', '%d-%b-%y', '%d/%b/%y']:
        try:
            return datetime.strptime(date_str, fmt)
        except:
            continue
    return None

def calculate_non_operating_days(bus):
    """Calculate non-operating days for a bus across different time periods"""
    try:
        # Get the worksheet data for this bus
        data, source = get_worksheet_data("All", bus)

        if not data or len(data) < 2:  # Need at least header + 1 data row
            print(f"⚠️ No data found for bus {bus}")
            return None

        # Extract date and odometer data
        # Column B (index 1) is date and column M (index 12) is odometer
        date_odometer_map = {}

        for row in data[1:]:  # Skip header
            if len(row) <= 12:  # Skip rows that don't have enough columns
                continue

            date_str = row[1].strip()
            odometer_str = row[12].strip()

            if not date_str or not odometer_str:
                continue

            parsed_date = parse_date(date_str)
            if not parsed_date:
                continue

            try:
                odometer_value = int(odometer_str)
                # Use date as key (without time) to group multiple entries on same date
                date_key = parsed_date.date()

                # For multiple entries on same date, keep the highest odometer value
                if date_key in date_odometer_map:
                    date_odometer_map[date_key] = max(date_odometer_map[date_key], odometer_value)
                else:
                    date_odometer_map[date_key] = odometer_value

            except ValueError:
                continue

        if not date_odometer_map:
            print(f"⚠️ No valid date-odometer pairs found for bus {bus}")
            return None

        # Sort unique dates in ascending order
        sorted_dates = sorted(date_odometer_map.keys())

        # Identify non-operating days (only between dates, not on the same date)
        non_operating_days = []

        # Find date ranges where bus was not operating
        for i in range(1, len(sorted_dates)):
            current_date = sorted_dates[i]
            prev_date = sorted_dates[i-1]

            # Get odometer values
            current_odo = date_odometer_map[current_date]
            prev_odo = date_odometer_map[prev_date]

            # If odometer value didn't change between different dates, bus didn't operate on current date
            if current_odo == prev_odo and current_date != prev_date:
                non_operating_days.append(current_date)

            # Check for missing days between consecutive dates
            date_diff = (current_date - prev_date).days
            if date_diff > 1:
                # For all days in between, mark as non-operating
                for day_offset in range(1, date_diff):
                    missing_date = prev_date + timedelta(days=day_offset)
                    non_operating_days.append(missing_date)

        # Count non-operating days in different periods
        today = datetime.now().date()
        one_week_ago_date = (today - timedelta(days=7)).date()
        one_month_ago_date = (today - timedelta(days=30)).date()
        one_year_ago_date = (today - timedelta(days=365)).date()

        # Filter non-operating days for each time period
        week_non_op_days = [d for d in non_operating_days if d >= one_week_ago_date and d <= today]
        month_non_op_days = [d for d in non_operating_days if d >= one_month_ago_date and d <= today]
        year_non_op_days = [d for d in non_operating_days if d >= one_year_ago_date and d <= today]

        week_non_op = len(week_non_op_days)
        month_non_op = len(month_non_op_days)
        year_non_op = len(year_non_op_days)

        # Calculate results
        with results_lock:
            bus_results[bus] = {
                'week_non_op': min(week_non_op, 7),  # Can't have more than 7 non-op days in a week
                'month_non_op': min(month_non_op, 30),  # Cap at 30 for a month
                'year_non_op': min(year_non_op, 365)  # Cap at 365 for a year
            }

        print(f"✅ Bus {bus}: Week({week_non_op}), Month({month_non_op}), Year({year_non_op}) non-operating days")
        return True

    except Exception as e:
        print(f"❌ Error processing bus {bus}: {e}")
        return None

# Process buses in parallel batches
MAX_WORKERS = 5  # Adjust based on API limits
BATCH_SIZE = 10  # Process this many buses at once

def process_buses_in_batches(buses):
    """Process buses in parallel batches"""
    total = len(buses)
    completed = 0

    bus_chunks = chunk_list(list(buses), BATCH_SIZE)

    for chunk in bus_chunks:
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(chunk))) as executor:
            futures = {executor.submit(calculate_non_operating_days, bus): bus for bus in chunk}
            for future in concurrent.futures.as_completed(futures):
                bus = futures[future]
                try:
                    result = future.result()
                    completed += 1
                    if completed % 10 == 0 or completed == total:
                        print(f"Progress: {completed}/{total} buses processed ({completed/total:.1%})")
                except Exception as e:
                    print(f"⚠️ Error processing bus {bus}: {e}")

        # Small delay between batches to avoid API rate limits
        time.sleep(1)

    print(f"✅ Completed processing all {total} buses")

# Process all buses in batches
if bus_set:
    process_buses_in_batches(bus_set)

# === STEP 4: UPDATE REPORT ===
print("\n📝 STEP 4: Updating report...")

# Prepare data for batch update
updated_rows = []
for row_idx, bus in row_map.items():
    result = bus_results.get(bus, {})

    week_non_op = result.get('week_non_op', '')
    month_non_op = result.get('month_non_op', '')
    year_non_op = result.get('year_non_op', '')

    updated_rows.append([week_non_op, month_non_op, year_non_op])

# Prepare batch update
if updated_rows:
    # Split into manageable chunks
    MAX_ROWS_PER_UPDATE = 1000  # Adjust based on API limits
    row_chunks = chunk_list(updated_rows, MAX_ROWS_PER_UPDATE)

    for i, chunk in enumerate(row_chunks):
        start_row = 2 + i * MAX_ROWS_PER_UPDATE
        end_row = start_row + len(chunk) - 1
        range_to_update = f"AH{start_row}:AJ{end_row}"

        Report_EO1_Worksheet.update(range_to_update, chunk)
        print(f"✅ Updated rows {start_row}-{end_row} in Report_EO1")

        # Small delay between batch updates
        if i < len(row_chunks) - 1:
            time.sleep(1)

    print(f"✅ All {len(updated_rows)} rows updated successfully in Report_EO1!")
else:
    print("❌ No updates made to Report_EO1.")

print("🎉 Process finished successfully!")

🚀 Initializing performance optimizations...

🧹 STEP 1: Clearing data...

📅 STEP 2: Getting current date...

🚌 STEP 3: Processing buses...
🔎 Total buses to process: 83
Sample buses: ['587', '475', '515', '486', '999']
📑 Found worksheet 587 in Odometer_SVP
❌ Error processing bus 587: 'datetime.date' object has no attribute 'date'
📑 Found worksheet 475 in Odometer_SVP
❌ Error processing bus 475: 'datetime.date' object has no attribute 'date'
📑 Found worksheet 515 in Odometer_SVP
❌ Error processing bus 515: 'datetime.date' object has no attribute 'date'
📑 Found worksheet 486 in Odometer_SVP
❌ Error processing bus 486: 'datetime.date' object has no attribute 'date'
📑 Found worksheet 999 in Odometer_BT
❌ Error processing bus 999: 'datetime.date' object has no attribute 'date'
📑 Found worksheet 578 in Odometer_SVP
❌ Error processing bus 578: 'datetime.date' object has no attribute 'date'
📑 Found worksheet 440 in Odometer_SVP
❌ Error processing bus 440: 'datetime.date' object has no attribute 

<ipython-input-16-632ae05fe42e>:308: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  Report_EO1_Worksheet.update(range_to_update, chunk)


✅ Updated rows 2-84 in Report_EO1
✅ All 83 rows updated successfully in Report_EO1!
🎉 Process finished successfully!


In [17]:
#R6 Deepseek

from datetime import datetime, timedelta
import re
import time
from collections import defaultdict
import functools
import concurrent.futures
import threading

# === PERFORMANCE OPTIMIZATION SETUP ===
print("🚀 Initializing performance optimizations...")

# Cache for worksheet data to reduce API calls
data_cache = {}
cache_lock = threading.Lock()

# LRU Cache decorator for expensive operations
def lru_cache_with_expiry(maxsize=128, typed=False, ttl=600):
    """LRU Cache with expiry time"""
    def decorator(func):
        cache_dict = {}
        cache_times = {}
        cache_lock = threading.Lock()

        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            key = str(args) + str(kwargs)
            current_time = time.time()

            with cache_lock:
                # Check if key exists and not expired
                if key in cache_dict and current_time - cache_times[key] < ttl:
                    return cache_dict[key]

                # Call the function and cache result
                result = func(*args, **kwargs)
                cache_dict[key] = result
                cache_times[key] = current_time

                # Remove oldest entries if cache is too large
                if len(cache_dict) > maxsize:
                    oldest_key = min(cache_times, key=cache_times.get)
                    cache_dict.pop(oldest_key)
                    cache_times.pop(oldest_key)

                return result
        return wrapper
    return decorator

# Batch processing helper functions
def chunk_list(lst, chunk_size):
    """Split list into chunks of specified size"""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

# === MAIN SCRIPT ===

# === STEP 1: CLEAR DATA ===
print("\n🧹 STEP 1: Clearing existing data...")
clear_range = ['AH2:AJ200']
Report_EO1_Worksheet.batch_clear(clear_range)
print("✅ Cleared columns AH-AJ in rows 2-200")

# === STEP 2: GET DATE RANGES ===
print("\n📅 STEP 2: Calculating date ranges...")
current_date = datetime.now()
date_formats = ['%d,%b%y', '%d-%b-%y', '%d/%b/%y']  # Supported date formats

# Calculate reference dates
week_ago = current_date - timedelta(days=7)
month_ago = current_date - timedelta(days=30)
year_ago = current_date - timedelta(days=365)

print(f"Current date: {current_date.strftime('%d,%b%y')}")
print(f"Last week: {week_ago.strftime('%d,%b%y')}")
print(f"Last month: {month_ago.strftime('%d,%b%y')}")
print(f"Last year: {year_ago.strftime('%d,%b%y')}")

# === STEP 3: PROCESS BUS DATA ===
print("\n🚌 STEP 3: Processing bus data...")

# Get main data from Report_EO1
main_data = Report_EO1_Worksheet.get_all_values()
header, rows = main_data[0], main_data[1:]

# Map each row to bus
row_map = {}
bus_set = set()
for i, row in enumerate(rows, start=2):
    try:
        if len(row) > 1:  # Make sure row has at least 2 columns
            bus = re.sub(r'[^a-zA-Z0-9]', '', str(row[1]).strip()).upper()  # Clean bus name
            if bus:
                row_map[i] = bus
                bus_set.add(bus)
    except Exception as e:
        print(f"⚠️ Error processing row {i}: {e}")
        continue

print(f"🔎 Total buses to process: {len(bus_set)}")
if bus_set:
    print("Sample buses:", list(bus_set)[:5])  # Log first 5 buses

# Initialize result storage with thread safety
non_op_days_map = defaultdict(dict)  # {bus: {'week': X, 'month': Y, 'year': Z}}
map_lock = threading.Lock()

def parse_date(date_str):
    """Try parsing date with multiple formats"""
    for fmt in date_formats:
        try:
            return datetime.strptime(date_str, fmt)
        except ValueError:
            continue
    return None

def get_bus_odometer_data(bus):
    """Retrieve and clean odometer data for a bus"""
    try:
        # Try to find the bus in all spreadsheets
        for spreadsheet in odometer_spreadsheets:
            try:
                sheet = spreadsheet.worksheet(bus)
                data = sheet.get_all_values()
                if data:
                    # Process data: date, odometer reading (assuming column 1 is date, column 12 is odometer)
                    clean_data = []
                    for row in data[1:]:  # Skip header
                        if len(row) >= 12:
                            date_str = row[1].strip()
                            odo_str = row[11].strip()  # Assuming column L (12th col) is odometer
                            if date_str and odo_str and odo_str.isdigit():
                                date = parse_date(date_str)
                                if date:
                                    clean_data.append((date, int(odo_str)))
                    return clean_data
            except:
                continue
        return []
    except Exception as e:
        print(f"⚠️ Error getting data for bus {bus}: {e}")
        return []

def calculate_non_operating_days(bus_data, current_date):
    """Calculate non-operating days for different periods"""
    if not bus_data:
        return {'week': 0, 'month': 0, 'year': 0}

    # Sort data by date
    bus_data.sort(key=lambda x: x[0])

    # Initialize counters
    non_op_week = 0
    non_op_month = 0
    non_op_year = 0

    # Track all dates in the data
    date_set = {entry[0] for entry in bus_data}
    min_date = min(bus_data[0][0], current_date - timedelta(days=365))
    max_date = current_date

    # Method 1: Same odometer on consecutive days
    for i in range(1, len(bus_data)):
        prev_date, prev_odo = bus_data[i-1]
        curr_date, curr_odo = bus_data[i]

        if curr_odo == prev_odo:
            # Bus didn't operate on curr_date
            days_diff = (curr_date - prev_date).days - 1
            if days_diff >= 0:
                # Check which periods this non-op day falls into
                for single_date in [prev_date + timedelta(days=d+1) for d in range(days_diff)]:
                    if single_date > current_date:
                        continue
                    if single_date >= week_ago:
                        non_op_week += 1
                    if single_date >= month_ago:
                        non_op_month += 1
                    if single_date >= year_ago:
                        non_op_year += 1

    # Method 2: Missing dates between entries
    all_dates = [entry[0] for entry in bus_data]
    for i in range(1, len(all_dates)):
        prev_date = all_dates[i-1]
        curr_date = all_dates[i]
        date_diff = (curr_date - prev_date).days

        if date_diff > 1:
            # Missing days between prev_date and curr_date
            for day in range(1, date_diff):
                missing_date = prev_date + timedelta(days=day)
                if missing_date > current_date:
                    continue
                # Check which periods this missing day falls into
                if missing_date >= week_ago:
                    non_op_week += 1
                if missing_date >= month_ago:
                    non_op_month += 1
                if missing_date >= year_ago:
                    non_op_year += 1

    return {
        'week': non_op_week,
        'month': non_op_month,
        'year': non_op_year
    }

def process_bus(bus):
    """Process a single bus to calculate non-operating days"""
    try:
        # Get odometer data for this bus
        bus_data = get_bus_odometer_data(bus)

        # Calculate non-operating days
        non_op_days = calculate_non_operating_days(bus_data, current_date)

        # Store results
        with map_lock:
            non_op_days_map[bus] = non_op_days

        print(f"✅ Processed {bus}: {non_op_days['week']}w/{non_op_days['month']}m/{non_op_days['year']}y")
        return True
    except Exception as e:
        print(f"⚠️ Error processing bus {bus}: {e}")
        return False

# Process buses in parallel
MAX_WORKERS = 5  # Adjust based on API limits
BATCH_SIZE = 10  # Process this many buses at once

def process_buses_in_batches(buses):
    """Process buses in parallel batches"""
    total = len(buses)
    completed = 0

    bus_chunks = chunk_list(list(buses), BATCH_SIZE)

    for chunk in bus_chunks:
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(chunk))) as executor:
            futures = {executor.submit(process_bus, bus): bus for bus in chunk}
            for future in concurrent.futures.as_completed(futures):
                bus = futures[future]
                try:
                    result = future.result()
                    completed += 1
                    if completed % 10 == 0 or completed == total:
                        print(f"Progress: {completed}/{total} buses processed ({completed/total:.1%})")
                except Exception as e:
                    print(f"⚠️ Error processing bus {bus}: {e}")

        # Small delay between batches to avoid API rate limits
        time.sleep(1)

    print(f"✅ Completed processing all {total} buses")

# Process all buses in batches
if bus_set:
    process_buses_in_batches(bus_set)

# === STEP 4: UPDATE REPORT ===
print("\n📝 STEP 4: Updating report worksheet...")

# Prepare update data
update_data = []
for row_idx, bus in row_map.items():
    non_op_days = non_op_days_map.get(bus, {'week': 0, 'month': 0, 'year': 0})
    update_data.append([
        non_op_days['week'],
        non_op_days['month'],
        non_op_days['year']
    ])

# Update worksheet in batches
if update_data:
    MAX_ROWS_PER_UPDATE = 100  # Adjust based on API limits
    row_chunks = chunk_list(update_data, MAX_ROWS_PER_UPDATE)

    for i, chunk in enumerate(row_chunks):
        start_row = 2 + i * MAX_ROWS_PER_UPDATE
        end_row = start_row + len(chunk) - 1
        range_to_update = f"AH{start_row}:AJ{end_row}"

        Report_EO1_Worksheet.update(range_to_update, chunk)
        print(f"✅ Updated rows {start_row}-{end_row} in Report_EO1")

        # Small delay between batch updates
        if i < len(row_chunks) - 1:
            time.sleep(1)

    print(f"✅ All {len(update_data)} rows updated successfully in Report_EO1!")
else:
    print("❌ No updates made to Report_EO1.")

print("🎉 Script execution completed successfully!")

🚀 Initializing performance optimizations...

🧹 STEP 1: Clearing existing data...
✅ Cleared columns AH-AJ in rows 2-200

📅 STEP 2: Calculating date ranges...
Current date: 16,May25
Last week: 09,May25
Last month: 16,Apr25
Last year: 16,May24

🚌 STEP 3: Processing bus data...
🔎 Total buses to process: 83
Sample buses: ['587', '475', '515', '486', '999']
✅ Processed 587: 0w/0m/0y
✅ Processed 515: 0w/0m/0y
✅ Processed 486: 0w/0m/0y
✅ Processed 475: 0w/0m/0y
✅ Processed 578: 0w/0m/0y
✅ Processed 440: 0w/0m/0y
✅ Processed 508: 0w/0m/0y
✅ Processed 999: 0w/0m/0y
✅ Processed 471: 0w/0m/0y
✅ Processed 561: 0w/0m/0y
Progress: 10/83 buses processed (12.0%)
✅ Processed 463: 0w/0m/0y
✅ Processed 613: 0w/0m/0y
✅ Processed 604: 0w/0m/0y
✅ Processed 460: 0w/0m/0y
✅ Processed 462: 0w/0m/0y
✅ Processed 529: 0w/0m/0y
✅ Processed 437: 0w/0m/0y
✅ Processed 563: 0w/0m/0y
✅ Processed 489: 0w/0m/0y
✅ Processed 493: 0w/0m/0y
Progress: 20/83 buses processed (24.1%)
✅ Processed 521: 0w/0m/0y
✅ Processed 444: 0w/

<ipython-input-17-bbc726604f3f>:284: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  Report_EO1_Worksheet.update(range_to_update, chunk)


✅ Updated rows 2-84 in Report_EO1
✅ All 83 rows updated successfully in Report_EO1!
🎉 Script execution completed successfully!


In [18]:
#R7

from datetime import datetime, timedelta
import re
import time
from collections import defaultdict
import functools
import concurrent.futures
import threading

# === PERFORMANCE OPTIMIZATION SETUP ===
print("🚀 Initializing performance optimizations...")

# Cache for worksheet data to reduce API calls
data_cache = {}
cache_lock = threading.Lock()

# === MAIN SCRIPT ===

# === STEP 1: CLEAR DATA ===
print("\n🧹 STEP 1: Clearing existing data...")
clear_range = ['AH2:AJ200']
Report_EO1_Worksheet.batch_clear(clear_range)
print("✅ Cleared columns AH-AJ in rows 2-200")

# === STEP 2: GET DATE RANGES ===
print("\n📅 STEP 2: Calculating date ranges...")
current_date = datetime.now()
date_formats = ['%d,%b%y', '%d-%b-%y', '%d/%b/%y', '%d,%b %y', '%d %b %y']  # More date formats

# Calculate reference dates
week_ago = current_date - timedelta(days=7)
month_ago = current_date - timedelta(days=30)
year_ago = current_date - timedelta(days=365)

print(f"Current date: {current_date.strftime('%d,%b%y')}")
print(f"Last week: {week_ago.strftime('%d,%b%y')}")
print(f"Last month: {month_ago.strftime('%d,%b%y')}")
print(f"Last year: {year_ago.strftime('%d,%b%y')}")

# === STEP 3: PROCESS BUS DATA ===
print("\n🚌 STEP 3: Processing bus data...")

# Get main data from Report_EO1
main_data = Report_EO1_Worksheet.get_all_values()
header, rows = main_data[0], main_data[1:]

# Map each row to bus
row_map = {}
bus_set = set()
for i, row in enumerate(rows, start=2):
    try:
        if len(row) > 1:  # Make sure row has at least 2 columns
            bus = re.sub(r'[^a-zA-Z0-9]', '', str(row[1]).strip()).upper()  # Clean bus name
            if bus:
                row_map[i] = bus
                bus_set.add(bus)
    except Exception as e:
        print(f"⚠️ Error processing row {i}: {e}")
        continue

print(f"🔎 Total buses to process: {len(bus_set)}")
if bus_set:
    print("Sample buses:", list(bus_set)[:5])  # Log first 5 buses

# Initialize result storage with thread safety
non_op_days_map = defaultdict(lambda: {'week': 0, 'month': 0, 'year': 0})
map_lock = threading.Lock()

def parse_date(date_str):
    """Try parsing date with multiple formats"""
    date_str = date_str.strip().replace(' ', '')  # Clean the date string
    for fmt in date_formats:
        try:
            return datetime.strptime(date_str, fmt).date()
        except ValueError:
            continue
    return None

def get_bus_odometer_data(bus):
    """Retrieve and clean odometer data for a bus"""
    try:
        # Try to find the bus in all spreadsheets
        for spreadsheet in odometer_spreadsheets:
            try:
                sheet = spreadsheet.worksheet(bus)
                data = sheet.get_all_values()
                if data and len(data) > 1:  # Has at least header + 1 row
                    clean_data = []
                    for row in data[1:]:  # Skip header
                        if len(row) >= 13:  # Ensure row has column M
                            date_str = row[1].strip()   # Column B (date)
                            odo_str = row[12].strip()  # Column M (odometer)

                            # Clean odometer value (remove commas, etc.)
                            odo_str = ''.join(c for c in odo_str if c.isdigit())

                            if date_str and odo_str and odo_str.isdigit():
                                date = parse_date(date_str)
                                if date:
                                    clean_data.append((date, int(odo_str)))
                    return clean_data
            except Exception as e:
                print(f"⚠️ Worksheet error for bus {bus}: {str(e)}")
                continue
        return []
    except Exception as e:
        print(f"⚠️ Error getting data for bus {bus}: {e}")
        return []

def calculate_non_operating_days(bus_data, current_date):
    """Calculate non-operating days for different periods"""
    if not bus_data or len(bus_data) < 2:
        return {'week': 0, 'month': 0, 'year': 0}

    # Convert current_date to date object for comparison
    current_date = current_date.date()
    week_ago_date = (current_date - timedelta(days=7))
    month_ago_date = (current_date - timedelta(days=30))
    year_ago_date = (current_date - timedelta(days=365))

    # Sort data by date
    bus_data.sort()

    # Initialize counters
    non_op_days = {'week': 0, 'month': 0, 'year': 0}

    # Method 1: Same odometer on consecutive days
    for i in range(1, len(bus_data)):
        prev_date, prev_odo = bus_data[i-1]
        curr_date, curr_odo = bus_data[i]

        if curr_odo == prev_odo:
            # The day after prev_date is non-operating
            non_op_date = prev_date + timedelta(days=1)

            # Only count if it's before current_date and matches curr_date
            if non_op_date < curr_date and non_op_date <= current_date:
                # Check which periods this non-op day falls into
                if non_op_date >= week_ago_date:
                    non_op_days['week'] += 1
                if non_op_date >= month_ago_date:
                    non_op_days['month'] += 1
                if non_op_date >= year_ago_date:
                    non_op_days['year'] += 1

    # Method 2: Missing dates between entries
    all_dates = [d for d, _ in bus_data]
    for i in range(1, len(all_dates)):
        prev_date = all_dates[i-1]
        curr_date = all_dates[i]
        days_diff = (curr_date - prev_date).days

        if days_diff > 1:
            # For each missing day between entries
            for day in range(1, days_diff):
                missing_date = prev_date + timedelta(days=day)
                if missing_date > current_date:
                    continue

                # Check which periods this missing day falls into
                if missing_date >= week_ago_date:
                    non_op_days['week'] += 1
                if missing_date >= month_ago_date:
                    non_op_days['month'] += 1
                if missing_date >= year_ago_date:
                    non_op_days['year'] += 1

    return non_op_days

def process_bus(bus):
    """Process a single bus to calculate non-operating days"""
    try:
        # Get odometer data for this bus
        bus_data = get_bus_odometer_data(bus)

        if not bus_data:
            print(f"⚠️ No valid odometer data found for {bus}")
            return False

        # Calculate non-operating days
        non_op_days = calculate_non_operating_days(bus_data, current_date)

        # Store results
        with map_lock:
            non_op_days_map[bus] = non_op_days

        print(f"✅ {bus}: {non_op_days['week']}w/{non_op_days['month']}m/{non_op_days['year']}y (Total entries: {len(bus_data)})")
        return True
    except Exception as e:
        print(f"⚠️ Error processing bus {bus}: {e}")
        return False

# Process buses in parallel
MAX_WORKERS = 5
BATCH_SIZE = 10

def process_buses_in_batches(buses):
    """Process buses in parallel batches"""
    total = len(buses)
    completed = 0

    bus_chunks = chunk_list(list(buses), BATCH_SIZE)

    for chunk in bus_chunks:
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(chunk))) as executor:
            futures = {executor.submit(process_bus, bus): bus for bus in chunk}
            for future in concurrent.futures.as_completed(futures):
                bus = futures[future]
                try:
                    result = future.result()
                    completed += 1
                    if completed % 10 == 0 or completed == total:
                        print(f"Progress: {completed}/{total} buses processed ({completed/total:.1%})")
                except Exception as e:
                    print(f"⚠️ Error processing bus {bus}: {e}")

        # Small delay between batches
        time.sleep(1)

    print(f"✅ Completed processing all {total} buses")

# Process all buses in batches
if bus_set:
    process_buses_in_batches(bus_set)

# === STEP 4: UPDATE REPORT ===
print("\n📝 STEP 4: Updating report worksheet...")

# Prepare update data
update_data = []
for row_idx, bus in row_map.items():
    non_op_days = non_op_days_map.get(bus, {'week': 0, 'month': 0, 'year': 0})
    update_data.append([
        non_op_days['week'],
        non_op_days['month'],
        non_op_days['year']
    ])

# Update worksheet in batches
if update_data:
    MAX_ROWS_PER_UPDATE = 100
    row_chunks = chunk_list(update_data, MAX_ROWS_PER_UPDATE)

    for i, chunk in enumerate(row_chunks):
        start_row = 2 + i * MAX_ROWS_PER_UPDATE
        end_row = start_row + len(chunk) - 1
        range_to_update = f"AH{start_row}:AJ{end_row}"

        Report_EO1_Worksheet.update(range_to_update, chunk)
        print(f"✅ Updated rows {start_row}-{end_row} in Report_EO1")

        if i < len(row_chunks) - 1:
            time.sleep(1)

    print(f"✅ All {len(update_data)} rows updated successfully!")
else:
    print("❌ No updates made - check if data was processed correctly")

print("🎉 Script execution completed successfully!")

🚀 Initializing performance optimizations...

🧹 STEP 1: Clearing existing data...
✅ Cleared columns AH-AJ in rows 2-200

📅 STEP 2: Calculating date ranges...
Current date: 16,May25
Last week: 09,May25
Last month: 16,Apr25
Last year: 16,May24

🚌 STEP 3: Processing bus data...
🔎 Total buses to process: 83
Sample buses: ['587', '475', '515', '486', '999']
⚠️ Worksheet error for bus 999: 999
✅ 587: 4w/14m/210y (Total entries: 177)
✅ 475: 0w/0m/257y (Total entries: 136)
✅ 515: 0w/0m/93y (Total entries: 389)
⚠️ Worksheet error for bus 999: 999
✅ 486: 0w/0m/247y (Total entries: 140)
✅ 578: 3w/15m/221y (Total entries: 199)
⚠️ Worksheet error for bus 561: 561
✅ 508: 6w/14m/203y (Total entries: 281)
✅ 440: 8w/26m/198y (Total entries: 301)
✅ 561: 0w/0m/59y (Total entries: 607)
✅ 471: 3w/11m/229y (Total entries: 183)
✅ 999: 0w/0m/77y (Total entries: 362)
Progress: 10/83 buses processed (12.0%)
✅ 604: 3w/15m/201y (Total entries: 235)
✅ 613: 0w/0m/46y (Total entries: 359)
⚠️ Worksheet error for bus 4

<ipython-input-18-7132d516e07f>:250: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  Report_EO1_Worksheet.update(range_to_update, chunk)


Now getting dates in AK, AL, AM columns

In [19]:
from datetime import datetime, timedelta
import re
import time
from collections import defaultdict
import functools
import concurrent.futures
import threading

# === PERFORMANCE OPTIMIZATION SETUP ===
print("🚀 Initializing performance optimizations...")

# Cache for worksheet data to reduce API calls
data_cache = {}
cache_lock = threading.Lock()

def chunk_list(lst, chunk_size):
    """Split list into chunks of specified size"""
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]

# === MAIN SCRIPT ===

# === STEP 1: CLEAR DATA ===
print("\n🧹 STEP 1: Clearing existing data...")
clear_range = ['AH2:AM200']  # Now clearing through column AM
Report_EO1_Worksheet.batch_clear(clear_range)
print("✅ Cleared columns AH-AM in rows 2-200")

# === STEP 2: GET DATE RANGES ===
print("\n📅 STEP 2: Calculating date ranges...")
current_date = datetime.now()
date_formats = ['%d,%b%y', '%d-%b-%y', '%d/%b/%y', '%d,%b %y', '%d %b %y']

# Calculate reference dates
week_ago = current_date - timedelta(days=7)
month_ago = current_date - timedelta(days=30)
year_ago = current_date - timedelta(days=365)

print(f"Current date: {current_date.strftime('%d,%b%y')}")
print(f"Last week: {week_ago.strftime('%d,%b%y')}")
print(f"Last month: {month_ago.strftime('%d,%b%y')}")
print(f"Last year: {year_ago.strftime('%d,%b%y')}")

# === STEP 3: PROCESS BUS DATA ===
print("\n🚌 STEP 3: Processing bus data...")

# Get main data from Report_EO1
main_data = Report_EO1_Worksheet.get_all_values()
header, rows = main_data[0], main_data[1:]

# Map each row to bus
row_map = {}
bus_set = set()
for i, row in enumerate(rows, start=2):
    try:
        if len(row) > 1:
            bus = re.sub(r'[^a-zA-Z0-9]', '', str(row[1]).strip()).upper()
            if bus:
                row_map[i] = bus
                bus_set.add(bus)
    except Exception as e:
        print(f"⚠️ Error processing row {i}: {e}")
        continue

print(f"🔎 Total buses to process: {len(bus_set)}")
if bus_set:
    print("Sample buses:", list(bus_set)[:5])

# Initialize result storage with thread safety
non_op_days_map = defaultdict(lambda: {
    'week': {'count': 0, 'dates': []},
    'month': {'count': 0, 'dates': []},
    'year': {'count': 0, 'dates': []}
})
map_lock = threading.Lock()

def parse_date(date_str):
    """Try parsing date with multiple formats"""
    date_str = date_str.strip().replace(' ', '')
    for fmt in date_formats:
        try:
            return datetime.strptime(date_str, fmt).date()
        except ValueError:
            continue
    return None

def format_date_list(dates):
    """Format list of dates as semicolon-separated string"""
    return ";".join(d.strftime('%d,%b%y') for d in sorted(dates))

def get_bus_odometer_data(bus):
    """Retrieve and clean odometer data for a bus"""
    try:
        for spreadsheet in odometer_spreadsheets:
            try:
                sheet = spreadsheet.worksheet(bus)
                data = sheet.get_all_values()
                if data and len(data) > 1:
                    clean_data = []
                    for row in data[1:]:
                        if len(row) >= 13:
                            date_str = row[1].strip()
                            odo_str = row[12].strip()
                            odo_str = ''.join(c for c in odo_str if c.isdigit())
                            if date_str and odo_str and odo_str.isdigit():
                                date = parse_date(date_str)
                                if date:
                                    clean_data.append((date, int(odo_str)))
                    return clean_data
            except Exception as e:
                continue
        return []
    except Exception as e:
        print(f"⚠️ Error getting data for bus {bus}: {e}")
        return []

def calculate_non_operating_days(bus_data, current_date):
    """Calculate non-operating days with actual dates"""
    result = {
        'week': {'count': 0, 'dates': []},
        'month': {'count': 0, 'dates': []},
        'year': {'count': 0, 'dates': []}
    }

    if not bus_data or len(bus_data) < 2:
        return result

    current_date = current_date.date()
    week_ago_date = (current_date - timedelta(days=7))
    month_ago_date = (current_date - timedelta(days=30))
    year_ago_date = (current_date - timedelta(days=365))

    bus_data.sort()

    # Method 1: Same odometer on consecutive days
    for i in range(1, len(bus_data)):
        prev_date, prev_odo = bus_data[i-1]
        curr_date, curr_odo = bus_data[i]

        if curr_odo == prev_odo:
            non_op_date = prev_date + timedelta(days=1)
            if non_op_date < curr_date and non_op_date <= current_date:
                # Add to appropriate periods
                if non_op_date >= week_ago_date:
                    result['week']['count'] += 1
                    result['week']['dates'].append(non_op_date)
                if non_op_date >= month_ago_date:
                    result['month']['count'] += 1
                    result['month']['dates'].append(non_op_date)
                if non_op_date >= year_ago_date:
                    result['year']['count'] += 1
                    result['year']['dates'].append(non_op_date)

    # Method 2: Missing dates between entries
    all_dates = [d for d, _ in bus_data]
    for i in range(1, len(all_dates)):
        prev_date = all_dates[i-1]
        curr_date = all_dates[i]
        days_diff = (curr_date - prev_date).days

        if days_diff > 1:
            for day in range(1, days_diff):
                missing_date = prev_date + timedelta(days=day)
                if missing_date > current_date:
                    continue

                if missing_date >= week_ago_date:
                    result['week']['count'] += 1
                    result['week']['dates'].append(missing_date)
                if missing_date >= month_ago_date:
                    result['month']['count'] += 1
                    result['month']['dates'].append(missing_date)
                if missing_date >= year_ago_date:
                    result['year']['count'] += 1
                    result['year']['dates'].append(missing_date)

    # Remove duplicates and sort dates
    for period in result:
        result[period]['dates'] = sorted(list(set(result[period]['dates'])))

    return result

def process_bus(bus):
    """Process a single bus to calculate non-operating days"""
    try:
        bus_data = get_bus_odometer_data(bus)

        if not bus_data:
            print(f"⚠️ No valid odometer data found for {bus}")
            return False

        non_op_days = calculate_non_operating_days(bus_data, current_date)

        with map_lock:
            non_op_days_map[bus] = non_op_days

        print(f"✅ {bus}: Week({non_op_days['week']['count']}): {format_date_list(non_op_days['week']['dates'])}")
        return True
    except Exception as e:
        print(f"⚠️ Error processing bus {bus}: {e}")
        return False

# Process buses in parallel
MAX_WORKERS = 5
BATCH_SIZE = 10

def process_buses_in_batches(buses):
    total = len(buses)
    completed = 0

    bus_chunks = chunk_list(list(buses), BATCH_SIZE)

    for chunk in bus_chunks:
        with concurrent.futures.ThreadPoolExecutor(max_workers=min(MAX_WORKERS, len(chunk))) as executor:
            futures = {executor.submit(process_bus, bus): bus for bus in chunk}
            for future in concurrent.futures.as_completed(futures):
                bus = futures[future]
                try:
                    result = future.result()
                    completed += 1
                    if completed % 10 == 0 or completed == total:
                        print(f"Progress: {completed}/{total} buses processed ({completed/total:.1%})")
                except Exception as e:
                    print(f"⚠️ Error processing bus {bus}: {e}")

        time.sleep(1)

    print(f"✅ Completed processing all {total} buses")

if bus_set:
    process_buses_in_batches(bus_set)

# === STEP 4: UPDATE REPORT ===
print("\n📝 STEP 4: Updating report worksheet...")

update_data = []
for row_idx, bus in row_map.items():
    non_op_days = non_op_days_map.get(bus, {
        'week': {'count': 0, 'dates': []},
        'month': {'count': 0, 'dates': []},
        'year': {'count': 0, 'dates': []}
    })

    update_data.append([
        non_op_days['week']['count'],
        non_op_days['month']['count'],
        non_op_days['year']['count'],
        format_date_list(non_op_days['week']['dates']),
        format_date_list(non_op_days['month']['dates']),
        format_date_list(non_op_days['year']['dates'])
    ])

if update_data:
    MAX_ROWS_PER_UPDATE = 100
    row_chunks = chunk_list(update_data, MAX_ROWS_PER_UPDATE)

    for i, chunk in enumerate(row_chunks):
        start_row = 2 + i * MAX_ROWS_PER_UPDATE
        end_row = start_row + len(chunk) - 1
        range_to_update = f"AH{start_row}:AM{end_row}"

        Report_EO1_Worksheet.update(range_to_update, chunk)
        print(f"✅ Updated rows {start_row}-{end_row}")

        if i < len(row_chunks) - 1:
            time.sleep(1)

    print(f"✅ All {len(update_data)} rows updated successfully!")
else:
    print("❌ No updates made - check if data was processed correctly")

print("🎉 Script execution completed successfully!")

🚀 Initializing performance optimizations...

🧹 STEP 1: Clearing existing data...
✅ Cleared columns AH-AM in rows 2-200

📅 STEP 2: Calculating date ranges...
Current date: 16,May25
Last week: 09,May25
Last month: 16,Apr25
Last year: 16,May24

🚌 STEP 3: Processing bus data...
🔎 Total buses to process: 83
Sample buses: ['587', '475', '515', '486', '999']
✅ 486: Week(0): ✅ 587: Week(4): 10,May25;12,May25;15,May25;16,May25

✅ 475: Week(0): 
✅ 515: Week(0): 
✅ 440: Week(8): 09,May25;10,May25;11,May25;12,May25;13,May25;14,May25;15,May25;16,May25
✅ 578: Week(3): 10,May25;11,May25;13,May25
✅ 508: Week(6): 09,May25;10,May25;12,May25;14,May25;15,May25;16,May25
✅ 561: Week(0): 
✅ 471: Week(3): 11,May25;14,May25;15,May25
✅ 999: Week(0): 
Progress: 10/83 buses processed (12.0%)
✅ 462: Week(6): 09,May25;10,May25;11,May25;12,May25;13,May25;15,May25
✅ 460: Week(8): 09,May25;10,May25;11,May25;12,May25;13,May25;14,May25;15,May25;16,May25
✅ 604: Week(3): 09,May25;10,May25;11,May25
✅ 463: Week(5): 09,May25

<ipython-input-19-afda1858d992>:261: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  Report_EO1_Worksheet.update(range_to_update, chunk)


✅ Updated rows 2-84
✅ All 83 rows updated successfully!
🎉 Script execution completed successfully!


In [20]:
#R10


from datetime import datetime, timedelta
import re
import time
from collections import defaultdict
import concurrent.futures
import threading

# === CONFIGURATION ===
DATE_FORMATS = ['%d,%b%y', '%d-%b-%y', '%d/%b/%y', '%d,%b %y', '%d %b %y']
MAX_WORKERS = 5
BATCH_SIZE = 10

# === INITIALIZATION ===
print("🚀 Initializing non-operating days calculator...")
current_date = datetime.now()

# === CORE FUNCTIONS ===
def parse_date(date_str):
    """Parse date from multiple possible formats"""
    date_str = date_str.strip().replace(' ', '')
    for fmt in DATE_FORMATS:
        try:
            return datetime.strptime(date_str, fmt).date()
        except ValueError:
            continue
    return None

def format_date_list(dates):
    """Format dates as DD,MonYY separated by semicolons"""
    return ";".join(d.strftime('%d,%b%y') for d in sorted(dates))

def get_bus_odometer_data(bus):
    """Retrieve odometer readings for a specific bus"""
    try:
        for spreadsheet in odometer_spreadsheets:
            try:
                sheet = spreadsheet.worksheet(bus)
                data = sheet.get_all_values()
                if data and len(data) > 1:
                    clean_data = []
                    for row in data[1:]:  # Skip header
                        if len(row) >= 13:  # Ensure column M exists
                            date_str = row[1].strip()   # Column B (date)
                            odo_str = row[12].strip()  # Column M (odometer)
                            odo_str = ''.join(c for c in odo_str if c.isdigit())
                            if date_str and odo_str and odo_str.isdigit():
                                date = parse_date(date_str)
                                if date:
                                    clean_data.append((date, int(odo_str)))
                    return clean_data
            except:
                continue
        return []
    except Exception as e:
        print(f"⚠️ Error getting data for {bus}: {e}")
        return []

def calculate_non_operating_days(bus_data, current_date):
    """Accurately identify non-operating days based on odometer changes"""
    current_date = current_date.date()
    periods = {
        'week': current_date - timedelta(days=7),
        'month': current_date - timedelta(days=30),
        'year': current_date - timedelta(days=365)
    }

    results = {p: {'count': 0, 'dates': set()} for p in periods}

    if not bus_data or len(bus_data) < 2:
        return results

    # Sort and remove duplicate dates
    bus_data.sort()
    unique_data = []
    last_date = None
    for date, odo in bus_data:
        if date != last_date:
            unique_data.append((date, odo))
            last_date = date

    # Find all dates when odometer increased (operating days)
    operating_dates = set()
    prev_odo = None
    for date, odo in unique_data:
        if prev_odo is not None and odo > prev_odo:
            operating_dates.add(date)
        prev_odo = odo

    # Generate complete date range to examine
    min_date = min(d for d, _ in unique_data)
    max_date = max(d for d, _ in unique_data)
    all_dates = [min_date + timedelta(days=x)
                for x in range((max_date - min_date).days + 1)]

    # Classify each date
    odometer_readings = {date: odo for date, odo in unique_data}
    prev_odo = None

    for date in all_dates:
        current_odo = odometer_readings.get(date)

        # Rule 1: Odometer increased - operating day
        if date in operating_dates:
            prev_odo = current_odo
            continue

        # Rule 2: Same odometer or missing date - non-operating
        if (current_odo is not None and prev_odo is not None and current_odo == prev_odo) or \
           (date not in odometer_readings):

            # Add to appropriate periods
            for period, cutoff in periods.items():
                if date >= cutoff:
                    results[period]['dates'].add(date)

        # Update previous odometer if available
        if current_odo is not None:
            prev_odo = current_odo

    # Finalize results
    for period in results:
        results[period]['dates'] = sorted(results[period]['dates'])
        results[period]['count'] = len(results[period]['dates'])

    return results

# === MAIN EXECUTION ===
def main():
    print("\n🧹 STEP 1: Clearing existing data...")
    Report_EO1_Worksheet.batch_clear(['AH2:AM200'])
    print("✅ Cleared columns AH-AM")

    print("\n📅 STEP 2: Loading bus list...")
    main_data = Report_EO1_Worksheet.get_all_values()
    row_map = {}
    for i, row in enumerate(main_data[1:], start=2):
        try:
            bus = re.sub(r'[^a-zA-Z0-9]', '', str(row[1]).strip()).upper()
            if bus:
                row_map[i] = bus
        except Exception as e:
            print(f"⚠️ Error processing row {i}: {e}")

    print(f"🔍 Found {len(row_map)} buses to process")

    print("\n🚌 STEP 3: Calculating non-operating days...")
    non_op_days_map = defaultdict(dict)
    lock = threading.Lock()

    def process_bus(bus, row_idx):
        try:
            bus_data = get_bus_odometer_data(bus)
            if not bus_data:
                print(f"⚠️ No data for {bus}")
                return

            results = calculate_non_operating_days(bus_data, current_date)

            with lock:
                non_op_days_map[row_idx] = {
                    'bus': bus,
                    'week': results['week'],
                    'month': results['month'],
                    'year': results['year']
                }

            print(f"✅ {bus}: {results['week']['count']}w/{results['month']['count']}m/{results['year']['count']}y")
        except Exception as e:
            print(f"⚠️ Failed to process {bus}: {e}")

    # Process in parallel batches
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        futures = []
        for row_idx, bus in row_map.items():
            futures.append(executor.submit(process_bus, bus, row_idx))
            if len(futures) >= BATCH_SIZE:
                concurrent.futures.wait(futures)
                futures = []
                time.sleep(1)  # Rate limiting

        if futures:
            concurrent.futures.wait(futures)

    print("\n📝 STEP 4: Updating worksheet...")
    update_data = []
    for row_idx, bus in row_map.items():
        results = non_op_days_map.get(row_idx, {
            'week': {'count': 0, 'dates': []},
            'month': {'count': 0, 'dates': []},
            'year': {'count': 0, 'dates': []}
        })

        update_data.append([
            results['week']['count'],
            results['month']['count'],
            results['year']['count'],
            format_date_list(results['week']['dates']),
            format_date_list(results['month']['dates']),
            format_date_list(results['year']['dates'])
        ])

    # Batch update
    for i in range(0, len(update_data), 100):
        batch = update_data[i:i+100]
        range_start = i + 2
        range_end = range_start + len(batch) - 1
        Report_EO1_Worksheet.update(
            f'AH{range_start}:AM{range_end}',
            batch
        )
        print(f"✅ Updated rows {range_start}-{range_end}")
        time.sleep(1)

    print("\n🎉 Done! Non-operating days calculation complete.")

if __name__ == '__main__':
    main()

🚀 Initializing non-operating days calculator...

🧹 STEP 1: Clearing existing data...
✅ Cleared columns AH-AM

📅 STEP 2: Loading bus list...
🔍 Found 83 buses to process

🚌 STEP 3: Calculating non-operating days...
✅ 442: 1w/13m/246y
✅ 444: 2w/8m/255y
✅ 441: 112w/135m/371y
✅ 440: 206w/224m/401y
✅ 457: 2w/12m/219y
✅ 459: 0w/5m/290y
✅ 460: 236w/259m/425y
✅ 462: 6w/29m/173y
✅ 469: 0w/11m/212y
✅ 471: 3w/11m/229y
✅ 474: 12w/26m/267y
✅ 473: 0w/0m/199y
✅ 476: 1281w/1290m/1454y
✅ 475: 0w/0m/260y
✅ 472: 5843w/5849m/5985y
✅ 489: 0w/0m/0y
✅ 486: 0w/0m/246y
✅ 504: 100w/115m/414y
✅ 495: 749w/751m/892y
✅ 508: 615w/623m/815y
✅ 518: 0w/0m/0y
✅ 521: 3w/13m/214y
✅ 511: 0w/15m/164y
✅ 514: 3w/9m/217y
✅ 515: 0w/0m/95y
✅ 530: 4w/15m/279y
✅ 523: 10892w/10911m/11142y
✅ 528: 0w/4m/227y
✅ 534: 2w/7m/143y
✅ 535: 2w/7m/90y
✅ 547: 234w/246m/499y
✅ 538: 1w/7m/163y
✅ 536: 1w/13m/127y
✅ 548: 3w/13m/209y
✅ 546: 2w/14m/199y
✅ 564: 0w/0m/0y
✅ 550: 244w/267m/529y
✅ 563: 2w/16m/225y
✅ 574: 0w/0m/114y
✅ 575: 0w/0m/114y
✅ 577

<ipython-input-20-fec457a84873>:210: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  Report_EO1_Worksheet.update(


APIError: APIError: [400]: Your input contains more than the maximum of 50000 characters in a single cell.